In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# AutoML SDK: AutoML tables regression model

## Installation

Install the latest version of AutoML SDK.

In [ ]:
! pip3 install google-cloud-automl


Install the Google *cloud-storage* library as well.

In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AutoML SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
     

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AutoML APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AutoML Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


#### Project ID

**If you don't know your project ID**, try to get your project ID using `gcloud` command by executing the second cell below.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AutoML. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AutoML. Not all regions provide support for all AutoML services. For the latest support per region, see [Region support for AutoML services]()

In [ ]:
REGION = 'us-central1' #@param {type: "string"}


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### Authenticate your GCP account

**If you are using AutoML Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AutoML notebook and run the cell, the cell knows to skip executing the authentication steps.*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AutoML, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME
    

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME
    

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

#### Import AutoML SDK

Import the AutoM SDK into our Python environment.

In [ ]:
import json


from google.cloud import automl_v1beta1 as automl
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Value


#### AutoML constants

Setup up the following constants for AutoML:

- `PARENT`: The AutoML location root path for dataset, model and endpoint resources.

In [ ]:
# AutoML location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


## Clients

The AutoML SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AutoML).

You will use several clients in this tutorial, so set them all up upfront.



In [ ]:
def automl_client():
    return automl.AutoMlClient()


def operations_client():
    return automl.AutoMlClient()._transport.operations_client


def prediction_client():
    return automl.PredictionServiceClient()


def tables_client():
    return automl.TablesClient(project=PROJECT_ID, region=REGION)


clients = {}
clients["automl"] = automl_client()
clients["operations"] = operations_client()
clients["prediction"] = prediction_client()
clients["tables"] = tables_client()


for client in clients.items():
    print(client)


In [ ]:
IMPORT_FILE = 'gs://cloud-ml-tables-data/bank-marketing.csv'


In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10 


*Example output*:
```
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome,Deposit
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,1
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,1
42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,1
58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,1
```



## Create a dataset

### Prepare data

### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request

In [ ]:
print(MessageToJson(
    automl.CreateDatasetRequest(
        dataset=automl.Dataset(
            display_name="bank_" + TIMESTAMP
        )
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "dataset": {
    "displayName": "bank_20210228161105"
  }
}
```



#### Call

In [ ]:
request = clients["tables"].create_dataset(
    dataset_display_name="bank_" + TIMESTAMP
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/TBL3707245895360708608",
  "displayName": "bank_20210228161105",
  "createTime": "2021-02-28T16:11:11.629918Z",
  "etag": "AB3BwFrEEPSvg_CecNSkJ7f0ROXZnZxzgAD5hxVGnRz7NlK5a_WrkIfS0tGbkJTeYFpQ",
  "tablesDatasetMetadata": {
    "statsUpdateTime": "1970-01-01T00:00:00Z"
  }
}
```



In [ ]:
# The full unique ID for the dataset
dataset_id = request.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)


### [projects.locations.datasets.importData](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

#### Request

In [ ]:
print(MessageToJson(
    automl.ImportDataRequest(
        name=dataset_id,
        input_config={"gcs_source": {"input_uris": [IMPORT_FILE]}}
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/TBL3707245895360708608",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://cloud-ml-tables-data/bank-marketing.csv"
      ]
    }
  }
}
```


#### Call

In [ ]:
request = clients["tables"].import_data(
    dataset_name=dataset_id,
    gcs_input_uris=[IMPORT_FILE]
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```

### [projects.locations.datasets.patch](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/patch)

#### Call

In [ ]:
request = clients["tables"].set_target_column(
    dataset_name=dataset_id,
    column_spec_display_name='Deposit'
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/TBL3707245895360708608",
  "displayName": "bank_20210228161105",
  "createTime": "2021-02-28T16:11:11.629918Z",
  "etag": "AB3BwFoGe9VDggahb_dNWX28GjKcqW9XDUytMYvTDIoFWTv2pbmfLrpMB2kge33oW2MY",
  "exampleCount": 45211,
  "tablesDatasetMetadata": {
    "primaryTableSpecId": "7767359156935196672",
    "targetColumnSpecId": "2361404080544284672",
    "statsUpdateTime": "2021-02-28T16:11:54.368477Z"
  }
}
```



## Train a model

### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

#### Call

In [ ]:
request = clients["tables"].create_model(
    model_display_name="bank_" + TIMESTAMP,
    dataset_name=dataset_id,
    train_budget_milli_node_hours=1*1000
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264"
}
```



In [ ]:
# The full unique ID for the training pipeline
model_id = result.name
# The short numeric ID for the training pipeline
model_short_id = model_id.split('/')[-1]

print(model_id)


## Evaluate the model

### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

#### Call

In [ ]:
request = clients["tables"].list_model_evaluations(
    model_name=model_id
)


#### Response

In [ ]:
evaluations_list = [
    json.loads(MessageToJson(me.__dict__["_pb"])) 
    for me in request.model_evaluation
]

print(json.dumps(evaluations_list, indent=2))
# The  evaluation slice
evaluation_slice = request.model_evaluation[0].name


*Example output*:
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/124023939996839080",
    "annotationSpecId": "not available",
    "createTime": "2021-02-28T17:24:31.800758Z",
    "evaluatedExampleCount": 4639,
    "classificationEvaluationMetrics": {
      "auPrc": 0.89604473,
      "confidenceMetricsEntry": [
        {
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.009267787,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.08771089,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.18867286,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.24711765,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.29667678,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.33910435,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.3722213,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.41141066,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.44597256,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.48088387,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.51082355,
          "recall": 0.9006251,
          "precision": 0.90688086,
          "f1Score": 0.90374213,
          "falsePositiveRate": 0.09247683,
          "truePositiveCount": "4178",
          "falsePositiveCount": "429",
          "falseNegativeCount": "461",
          "trueNegativeCount": "4210",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5395309,
          "recall": 0.8902781,
          "precision": 0.9145261,
          "f1Score": 0.9022392,
          "falsePositiveRate": 0.083207585,
          "truePositiveCount": "4130",
          "falsePositiveCount": "386",
          "falseNegativeCount": "509",
          "trueNegativeCount": "4253",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.569127,
          "recall": 0.88014656,
          "precision": 0.92083895,
          "f1Score": 0.90003306,
          "falsePositiveRate": 0.07566286,
          "truePositiveCount": "4083",
          "falsePositiveCount": "351",
          "falseNegativeCount": "556",
          "trueNegativeCount": "4288",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.59819055,
          "recall": 0.8700151,
          "precision": 0.9284564,
          "f1Score": 0.8982862,
          "falsePositiveRate": 0.06704031,
          "truePositiveCount": "4036",
          "falsePositiveCount": "311",
          "falseNegativeCount": "603",
          "trueNegativeCount": "4328",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6267493,
          "recall": 0.86009914,
          "precision": 0.9350832,
          "f1Score": 0.8960252,
          "falsePositiveRate": 0.059711143,
          "truePositiveCount": "3990",
          "falsePositiveCount": "277",
          "falseNegativeCount": "649",
          "trueNegativeCount": "4362",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6483357,
          "recall": 0.84996766,
          "precision": 0.9401526,
          "f1Score": 0.8927884,
          "falsePositiveRate": 0.05410649,
          "truePositiveCount": "3943",
          "falsePositiveCount": "251",
          "falseNegativeCount": "696",
          "trueNegativeCount": "4388",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.68742466,
          "recall": 0.8404829,
          "precision": 0.95051193,
          "f1Score": 0.8921176,
          "falsePositiveRate": 0.04375943,
          "truePositiveCount": "3899",
          "falsePositiveCount": "203",
          "falseNegativeCount": "740",
          "trueNegativeCount": "4436",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7178758,
          "recall": 0.83035135,
          "precision": 0.95630586,
          "f1Score": 0.8888889,
          "falsePositiveRate": 0.03793921,
          "truePositiveCount": "3852",
          "falsePositiveCount": "176",
          "falseNegativeCount": "787",
          "trueNegativeCount": "4463",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7454512,
          "recall": 0.8200043,
          "precision": 0.9620637,
          "f1Score": 0.8853718,
          "falsePositiveRate": 0.032334555,
          "truePositiveCount": "3804",
          "falsePositiveCount": "150",
          "falseNegativeCount": "835",
          "trueNegativeCount": "4489",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7702336,
          "recall": 0.8105195,
          "precision": 0.9685729,
          "f1Score": 0.8825255,
          "falsePositiveRate": 0.02629877,
          "truePositiveCount": "3760",
          "falsePositiveCount": "122",
          "falseNegativeCount": "879",
          "trueNegativeCount": "4517",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.79181457,
          "recall": 0.80038804,
          "precision": 0.972244,
          "f1Score": 0.87798536,
          "falsePositiveRate": 0.022849752,
          "truePositiveCount": "3713",
          "falsePositiveCount": "106",
          "falseNegativeCount": "926",
          "trueNegativeCount": "4533",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.80905926,
          "recall": 0.7902565,
          "precision": 0.9755189,
          "f1Score": 0.873169,
          "falsePositiveRate": 0.01983186,
          "truePositiveCount": "3666",
          "falsePositiveCount": "92",
          "falseNegativeCount": "973",
          "trueNegativeCount": "4547",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.82475233,
          "recall": 0.7803406,
          "precision": 0.97574127,
          "f1Score": 0.86716974,
          "falsePositiveRate": 0.019400733,
          "truePositiveCount": "3620",
          "falsePositiveCount": "90",
          "falseNegativeCount": "1019",
          "trueNegativeCount": "4549",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.84347624,
          "recall": 0.7702091,
          "precision": 0.97863597,
          "f1Score": 0.86200243,
          "falsePositiveRate": 0.01681397,
          "truePositiveCount": "3573",
          "falsePositiveCount": "78",
          "falseNegativeCount": "1066",
          "trueNegativeCount": "4561",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.8612723,
          "recall": 0.7602932,
          "precision": 0.9799944,
          "f1Score": 0.8562758,
          "falsePositiveRate": 0.015520587,
          "truePositiveCount": "3527",
          "falsePositiveCount": "72",
          "falseNegativeCount": "1112",
          "trueNegativeCount": "4567",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.87450475,
          "recall": 0.75016165,
          "precision": 0.9811108,
          "f1Score": 0.8502321,
          "falsePositiveRate": 0.014442768,
          "truePositiveCount": "3480",
          "falsePositiveCount": "67",
          "falseNegativeCount": "1159",
          "trueNegativeCount": "4572",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.8876167,
          "recall": 0.74003017,
          "precision": 0.9825415,
          "f1Score": 0.8442149,
          "falsePositiveRate": 0.013149385,
          "truePositiveCount": "3433",
          "falsePositiveCount": "61",
          "falseNegativeCount": "1206",
          "trueNegativeCount": "4578",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.89729506,
          "recall": 0.7303298,
          "precision": 0.98402554,
          "f1Score": 0.8384063,
          "falsePositiveRate": 0.011856004,
          "truePositiveCount": "3388",
          "falsePositiveCount": "55",
          "falseNegativeCount": "1251",
          "trueNegativeCount": "4584",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9069939,
          "recall": 0.72041386,
          "precision": 0.9858407,
          "f1Score": 0.8324823,
          "falsePositiveRate": 0.010347057,
          "truePositiveCount": "3342",
          "falsePositiveCount": "48",
          "falseNegativeCount": "1297",
          "trueNegativeCount": "4591",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.91544837,
          "recall": 0.7102824,
          "precision": 0.987118,
          "f1Score": 0.8261251,
          "falsePositiveRate": 0.009269239,
          "truePositiveCount": "3295",
          "falsePositiveCount": "43",
          "falseNegativeCount": "1344",
          "trueNegativeCount": "4596",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9246296,
          "recall": 0.700582,
          "precision": 0.9896468,
          "f1Score": 0.8203963,
          "falsePositiveRate": 0.007329166,
          "truePositiveCount": "3250",
          "falsePositiveCount": "34",
          "falseNegativeCount": "1389",
          "trueNegativeCount": "4605",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.932719,
          "recall": 0.6900194,
          "precision": 0.99071497,
          "f1Score": 0.8134689,
          "falsePositiveRate": 0.006466911,
          "truePositiveCount": "3201",
          "falsePositiveCount": "30",
          "falseNegativeCount": "1438",
          "trueNegativeCount": "4609",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.93940556,
          "recall": 0.6807502,
          "precision": 0.9921458,
          "f1Score": 0.80746615,
          "falsePositiveRate": 0.0053890925,
          "truePositiveCount": "3158",
          "falsePositiveCount": "25",
          "falseNegativeCount": "1481",
          "trueNegativeCount": "4614",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9436984,
          "recall": 0.6704031,
          "precision": 0.99234205,
          "f1Score": 0.8002058,
          "falsePositiveRate": 0.0051735286,
          "truePositiveCount": "3110",
          "falsePositiveCount": "24",
          "falseNegativeCount": "1529",
          "trueNegativeCount": "4615",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9486017,
          "recall": 0.6602716,
          "precision": 0.992547,
          "f1Score": 0.7930097,
          "falsePositiveRate": 0.004957965,
          "truePositiveCount": "3063",
          "falsePositiveCount": "23",
          "falseNegativeCount": "1576",
          "trueNegativeCount": "4616",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.95302504,
          "recall": 0.6501401,
          "precision": 0.9937397,
          "f1Score": 0.78603077,
          "falsePositiveRate": 0.0040957103,
          "truePositiveCount": "3016",
          "falsePositiveCount": "19",
          "falseNegativeCount": "1623",
          "trueNegativeCount": "4620",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.95584136,
          "recall": 0.6400086,
          "precision": 0.99530673,
          "f1Score": 0.7790606,
          "falsePositiveRate": 0.003017892,
          "truePositiveCount": "2969",
          "falsePositiveCount": "14",
          "falseNegativeCount": "1670",
          "trueNegativeCount": "4625",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.95881057,
          "recall": 0.6300927,
          "precision": 0.9955722,
          "f1Score": 0.7717492,
          "falsePositiveRate": 0.002802328,
          "truePositiveCount": "2923",
          "falsePositiveCount": "13",
          "falseNegativeCount": "1716",
          "trueNegativeCount": "4626",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96166563,
          "recall": 0.62082344,
          "precision": 0.9961951,
          "f1Score": 0.76494026,
          "falsePositiveRate": 0.0023712006,
          "truePositiveCount": "2880",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1759",
          "trueNegativeCount": "4628",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96420497,
          "recall": 0.61026084,
          "precision": 0.9961295,
          "f1Score": 0.75685066,
          "falsePositiveRate": 0.0023712006,
          "truePositiveCount": "2831",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1808",
          "trueNegativeCount": "4628",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96650803,
          "recall": 0.5999138,
          "precision": 0.996063,
          "f1Score": 0.7488228,
          "falsePositiveRate": 0.0023712006,
          "truePositiveCount": "2783",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1856",
          "trueNegativeCount": "4628",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96897614,
          "recall": 0.59042895,
          "precision": 0.9963623,
          "f1Score": 0.74147266,
          "falsePositiveRate": 0.002155637,
          "truePositiveCount": "2739",
          "falsePositiveCount": "10",
          "falseNegativeCount": "1900",
          "trueNegativeCount": "4629",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.97092766,
          "recall": 0.5807286,
          "precision": 0.99667037,
          "f1Score": 0.73385996,
          "falsePositiveRate": 0.0019400733,
          "truePositiveCount": "2694",
          "falsePositiveCount": "9",
          "falseNegativeCount": "1945",
          "trueNegativeCount": "4630",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9727514,
          "recall": 0.5699504,
          "precision": 0.9969834,
          "f1Score": 0.7252777,
          "falsePositiveRate": 0.0017245096,
          "truePositiveCount": "2644",
          "falsePositiveCount": "8",
          "falseNegativeCount": "1995",
          "trueNegativeCount": "4631",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9747563,
          "recall": 0.56046563,
          "precision": 0.99731493,
          "f1Score": 0.7176373,
          "falsePositiveRate": 0.001508946,
          "truePositiveCount": "2600",
          "falsePositiveCount": "7",
          "falseNegativeCount": "2039",
          "trueNegativeCount": "4632",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9766838,
          "recall": 0.549903,
          "precision": 0.9976535,
          "f1Score": 0.709005,
          "falsePositiveRate": 0.0012933821,
          "truePositiveCount": "2551",
          "falsePositiveCount": "6",
          "falseNegativeCount": "2088",
          "trueNegativeCount": "4633",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.97776866,
          "recall": 0.54063374,
          "precision": 0.99801034,
          "f1Score": 0.7013423,
          "falsePositiveRate": 0.0010778185,
          "truePositiveCount": "2508",
          "falsePositiveCount": "5",
          "falseNegativeCount": "2131",
          "trueNegativeCount": "4634",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9792647,
          "recall": 0.53050226,
          "precision": 0.9979724,
          "f1Score": 0.6927516,
          "falsePositiveRate": 0.0010778185,
          "truePositiveCount": "2461",
          "falsePositiveCount": "5",
          "falseNegativeCount": "2178",
          "trueNegativeCount": "4634",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9804138,
          "recall": 0.5203708,
          "precision": 0.99834573,
          "f1Score": 0.6841434,
          "falsePositiveRate": 0.0008622548,
          "truePositiveCount": "2414",
          "falsePositiveCount": "4",
          "falseNegativeCount": "2225",
          "trueNegativeCount": "4635",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9814273,
          "recall": 0.51045483,
          "precision": 0.9991561,
          "f1Score": 0.6757027,
          "falsePositiveRate": 0.0004311274,
          "truePositiveCount": "2368",
          "falsePositiveCount": "2",
          "falseNegativeCount": "2271",
          "trueNegativeCount": "4637",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98243207,
          "recall": 0.5005389,
          "precision": 0.9991394,
          "f1Score": 0.6669539,
          "falsePositiveRate": 0.0004311274,
          "truePositiveCount": "2322",
          "falsePositiveCount": "2",
          "falseNegativeCount": "2317",
          "trueNegativeCount": "4637",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9835001,
          "recall": 0.49019185,
          "precision": 0.9995604,
          "f1Score": 0.6577958,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2274",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2365",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9844133,
          "recall": 0.4804915,
          "precision": 0.9995516,
          "f1Score": 0.6490028,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2229",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2410",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98528725,
          "recall": 0.47079113,
          "precision": 0.99954236,
          "f1Score": 0.6400938,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2184",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2455",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98630506,
          "recall": 0.46001294,
          "precision": 0.9995316,
          "f1Score": 0.6300561,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2134",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2505",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9871925,
          "recall": 0.44988143,
          "precision": 0.9995211,
          "f1Score": 0.6204846,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2087",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2552",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9877509,
          "recall": 0.44082776,
          "precision": 0.99951124,
          "f1Score": 0.6118175,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2045",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2594",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9885219,
          "recall": 0.43048072,
          "precision": 0.9994995,
          "f1Score": 0.6017779,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1997",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2642",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9892192,
          "recall": 0.4199181,
          "precision": 0.9994869,
          "f1Score": 0.5913783,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1948",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2691",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9897657,
          "recall": 0.41000214,
          "precision": 0.9994745,
          "f1Score": 0.5814735,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1902",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2737",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9902156,
          "recall": 0.40051734,
          "precision": 0.99946207,
          "f1Score": 0.57186824,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1858",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2781",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.990671,
          "recall": 0.3901703,
          "precision": 0.9994478,
          "f1Score": 0.5612403,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1810",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2829",
          "trueNegativeCount": "4638",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99106044,
          "recall": 0.38025436,
          "precision": 1.0,
          "f1Score": 0.5509917,
          "truePositiveCount": "1764",
          "falseNegativeCount": "2875",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99142385,
          "recall": 0.36990732,
          "precision": 1.0,
          "f1Score": 0.5400472,
          "truePositiveCount": "1716",
          "falseNegativeCount": "2923",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.991821,
          "recall": 0.35999137,
          "precision": 1.0,
          "f1Score": 0.52940243,
          "truePositiveCount": "1670",
          "falseNegativeCount": "2969",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99219143,
          "recall": 0.35007545,
          "precision": 1.0,
          "f1Score": 0.5186013,
          "truePositiveCount": "1624",
          "falseNegativeCount": "3015",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99265563,
          "recall": 0.3401595,
          "precision": 1.0,
          "f1Score": 0.50764036,
          "truePositiveCount": "1578",
          "falseNegativeCount": "3061",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9930375,
          "recall": 0.33045915,
          "precision": 1.0,
          "f1Score": 0.49675956,
          "truePositiveCount": "1533",
          "falseNegativeCount": "3106",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9932301,
          "recall": 0.32054323,
          "precision": 1.0,
          "f1Score": 0.48547176,
          "truePositiveCount": "1487",
          "falseNegativeCount": "3152",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99349546,
          "recall": 0.31041172,
          "precision": 1.0,
          "f1Score": 0.47376212,
          "truePositiveCount": "1440",
          "falseNegativeCount": "3199",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.993766,
          "recall": 0.30006468,
          "precision": 1.0,
          "f1Score": 0.461615,
          "truePositiveCount": "1392",
          "falseNegativeCount": "3247",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9940096,
          "recall": 0.29014874,
          "precision": 1.0,
          "f1Score": 0.44979113,
          "truePositiveCount": "1346",
          "falseNegativeCount": "3293",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9942971,
          "recall": 0.27980167,
          "precision": 1.0,
          "f1Score": 0.4372579,
          "truePositiveCount": "1298",
          "falseNegativeCount": "3341",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9945464,
          "recall": 0.26988575,
          "precision": 1.0,
          "f1Score": 0.42505518,
          "truePositiveCount": "1252",
          "falseNegativeCount": "3387",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.994734,
          "recall": 0.2601854,
          "precision": 1.0,
          "f1Score": 0.41293192,
          "truePositiveCount": "1207",
          "falseNegativeCount": "3432",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9949861,
          "recall": 0.25026944,
          "precision": 1.0,
          "f1Score": 0.40034482,
          "truePositiveCount": "1161",
          "falseNegativeCount": "3478",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99523896,
          "recall": 0.2399224,
          "precision": 1.0,
          "f1Score": 0.38699582,
          "truePositiveCount": "1113",
          "falseNegativeCount": "3526",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99545944,
          "recall": 0.23043759,
          "precision": 1.0,
          "f1Score": 0.37456203,
          "truePositiveCount": "1069",
          "falseNegativeCount": "3570",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9956529,
          "recall": 0.22009054,
          "precision": 1.0,
          "f1Score": 0.36077738,
          "truePositiveCount": "1021",
          "falseNegativeCount": "3618",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9958653,
          "recall": 0.21039017,
          "precision": 1.0,
          "f1Score": 0.34764025,
          "truePositiveCount": "976",
          "falseNegativeCount": "3663",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99604857,
          "recall": 0.20025867,
          "precision": 1.0,
          "f1Score": 0.33369252,
          "truePositiveCount": "929",
          "falseNegativeCount": "3710",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99617386,
          "recall": 0.19012718,
          "precision": 1.0,
          "f1Score": 0.31950733,
          "truePositiveCount": "882",
          "falseNegativeCount": "3757",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9963188,
          "recall": 0.18042682,
          "precision": 1.0,
          "f1Score": 0.3056976,
          "truePositiveCount": "837",
          "falseNegativeCount": "3802",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99644965,
          "recall": 0.17029533,
          "precision": 1.0,
          "f1Score": 0.29102966,
          "truePositiveCount": "790",
          "falseNegativeCount": "3849",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99657893,
          "recall": 0.16059496,
          "precision": 1.0,
          "f1Score": 0.27674592,
          "truePositiveCount": "745",
          "falseNegativeCount": "3894",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99672806,
          "recall": 0.1502479,
          "precision": 1.0,
          "f1Score": 0.2612444,
          "truePositiveCount": "697",
          "falseNegativeCount": "3942",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9968369,
          "recall": 0.14033197,
          "precision": 1.0,
          "f1Score": 0.24612476,
          "truePositiveCount": "651",
          "falseNegativeCount": "3988",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9969655,
          "recall": 0.13041604,
          "precision": 1.0,
          "f1Score": 0.23073989,
          "truePositiveCount": "605",
          "falseNegativeCount": "4034",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9970963,
          "recall": 0.12028454,
          "precision": 1.0,
          "f1Score": 0.21473928,
          "truePositiveCount": "558",
          "falseNegativeCount": "4081",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9972023,
          "recall": 0.110799745,
          "precision": 1.0,
          "f1Score": 0.19949543,
          "truePositiveCount": "514",
          "falseNegativeCount": "4125",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9973033,
          "recall": 0.100668244,
          "precision": 1.0,
          "f1Score": 0.18292205,
          "truePositiveCount": "467",
          "falseNegativeCount": "4172",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99741936,
          "recall": 0.09053675,
          "precision": 1.0,
          "f1Score": 0.16604072,
          "truePositiveCount": "420",
          "falseNegativeCount": "4219",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9974875,
          "recall": 0.080620825,
          "precision": 1.0,
          "f1Score": 0.14921205,
          "truePositiveCount": "374",
          "falseNegativeCount": "4265",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9976044,
          "recall": 0.070273764,
          "precision": 1.0,
          "f1Score": 0.13131924,
          "truePositiveCount": "326",
          "falseNegativeCount": "4313",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99769753,
          "recall": 0.06014227,
          "precision": 1.0,
          "f1Score": 0.11346076,
          "truePositiveCount": "279",
          "falseNegativeCount": "4360",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99778664,
          "recall": 0.049795214,
          "precision": 1.0,
          "f1Score": 0.09486653,
          "truePositiveCount": "231",
          "falseNegativeCount": "4408",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9978742,
          "recall": 0.040525977,
          "precision": 1.0,
          "f1Score": 0.07789517,
          "truePositiveCount": "188",
          "falseNegativeCount": "4451",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.997992,
          "recall": 0.030394481,
          "precision": 1.0,
          "f1Score": 0.058995817,
          "truePositiveCount": "141",
          "falseNegativeCount": "4498",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9980989,
          "recall": 0.020262988,
          "precision": 1.0,
          "f1Score": 0.03972111,
          "truePositiveCount": "94",
          "falseNegativeCount": "4545",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99821925,
          "recall": 0.010347057,
          "precision": 1.0,
          "f1Score": 0.020482184,
          "truePositiveCount": "48",
          "falseNegativeCount": "4591",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9987167,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4639",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 1.0,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4639",
          "trueNegativeCount": "4639",
          "positionThreshold": 1
        }
      ],
      "confusionMatrix": {
        "row": [
          {
            "exampleCount": [
              3941,
              147
            ]
          },
          {
            "exampleCount": [
              293,
              258
            ]
          }
        ],
        "displayName": [
          "1",
          "2"
        ]
      },
      "auRoc": 0.94022393,
      "logLoss": 0.20077285
    }
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/542606150386485790",
    "annotationSpecId": "not available",
    "createTime": "2021-02-28T17:24:31.800758Z",
    "evaluatedExampleCount": 4088,
    "classificationEvaluationMetrics": {
      "auPrc": 0.9907691,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.8812244,
          "f1Score": 0.9368626,
          "falsePositiveRate": 1.0,
          "truePositiveCount": "4088",
          "falsePositiveCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.07797722,
          "recall": 0.9997554,
          "precision": 0.8815789,
          "f1Score": 0.9369555,
          "falsePositiveRate": 0.99637026,
          "truePositiveCount": "4087",
          "falsePositiveCount": "549",
          "falseNegativeCount": "1",
          "trueNegativeCount": "2",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.30998033,
          "recall": 0.9899706,
          "precision": 0.90254235,
          "f1Score": 0.94423705,
          "falsePositiveRate": 0.79310346,
          "truePositiveCount": "4047",
          "falsePositiveCount": "437",
          "falseNegativeCount": "41",
          "trueNegativeCount": "114",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.39478266,
          "recall": 0.9801859,
          "precision": 0.913797,
          "f1Score": 0.9458279,
          "falsePositiveRate": 0.6860254,
          "truePositiveCount": "4007",
          "falsePositiveCount": "378",
          "falseNegativeCount": "81",
          "trueNegativeCount": "173",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.46149504,
          "recall": 0.96991193,
          "precision": 0.9246735,
          "f1Score": 0.9467526,
          "falsePositiveRate": 0.5862069,
          "truePositiveCount": "3965",
          "falsePositiveCount": "323",
          "falseNegativeCount": "123",
          "trueNegativeCount": "228",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.51386535,
          "recall": 0.95988256,
          "precision": 0.93361884,
          "f1Score": 0.94656855,
          "falsePositiveRate": 0.50635207,
          "truePositiveCount": "3924",
          "falsePositiveCount": "279",
          "falseNegativeCount": "164",
          "trueNegativeCount": "272",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5659395,
          "recall": 0.94985324,
          "precision": 0.9406492,
          "f1Score": 0.9452288,
          "falsePositiveRate": 0.4446461,
          "truePositiveCount": "3883",
          "falsePositiveCount": "245",
          "falseNegativeCount": "205",
          "trueNegativeCount": "306",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6215324,
          "recall": 0.93982387,
          "precision": 0.94817376,
          "f1Score": 0.94398034,
          "falsePositiveRate": 0.38112524,
          "truePositiveCount": "3842",
          "falsePositiveCount": "210",
          "falseNegativeCount": "246",
          "trueNegativeCount": "341",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.65884733,
          "recall": 0.93003917,
          "precision": 0.9533601,
          "f1Score": 0.9415552,
          "falsePositiveRate": 0.33756804,
          "truePositiveCount": "3802",
          "falsePositiveCount": "186",
          "falseNegativeCount": "286",
          "trueNegativeCount": "365",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7143411,
          "recall": 0.9200098,
          "precision": 0.9623849,
          "f1Score": 0.9407204,
          "falsePositiveRate": 0.26678765,
          "truePositiveCount": "3761",
          "falsePositiveCount": "147",
          "falseNegativeCount": "327",
          "trueNegativeCount": "404",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7492718,
          "recall": 0.9099804,
          "precision": 0.96749026,
          "f1Score": 0.9378545,
          "falsePositiveRate": 0.22686026,
          "truePositiveCount": "3720",
          "falsePositiveCount": "125",
          "falseNegativeCount": "368",
          "trueNegativeCount": "426",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7795446,
          "recall": 0.9001957,
          "precision": 0.97303015,
          "f1Score": 0.93519694,
          "falsePositiveRate": 0.18511796,
          "truePositiveCount": "3680",
          "falsePositiveCount": "102",
          "falseNegativeCount": "408",
          "trueNegativeCount": "449",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7989856,
          "recall": 0.8899217,
          "precision": 0.97559667,
          "f1Score": 0.93079185,
          "falsePositiveRate": 0.16515426,
          "truePositiveCount": "3638",
          "falsePositiveCount": "91",
          "falseNegativeCount": "450",
          "trueNegativeCount": "460",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.82042044,
          "recall": 0.87989235,
          "precision": 0.9777113,
          "f1Score": 0.9262263,
          "falsePositiveRate": 0.14882033,
          "truePositiveCount": "3597",
          "falsePositiveCount": "82",
          "falseNegativeCount": "491",
          "trueNegativeCount": "469",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.83562684,
          "recall": 0.87010765,
          "precision": 0.9790807,
          "f1Score": 0.92138326,
          "falsePositiveRate": 0.13793103,
          "truePositiveCount": "3557",
          "falsePositiveCount": "76",
          "falseNegativeCount": "531",
          "trueNegativeCount": "475",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.85593396,
          "recall": 0.8600783,
          "precision": 0.98102677,
          "f1Score": 0.9165798,
          "falsePositiveRate": 0.123411976,
          "truePositiveCount": "3516",
          "falsePositiveCount": "68",
          "falseNegativeCount": "572",
          "trueNegativeCount": "483",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.86959773,
          "recall": 0.8500489,
          "precision": 0.9810841,
          "f1Score": 0.9108781,
          "falsePositiveRate": 0.1215971,
          "truePositiveCount": "3475",
          "falsePositiveCount": "67",
          "falseNegativeCount": "613",
          "trueNegativeCount": "484",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.88389677,
          "recall": 0.83977497,
          "precision": 0.98338586,
          "f1Score": 0.90592426,
          "falsePositiveRate": 0.10526316,
          "truePositiveCount": "3433",
          "falsePositiveCount": "58",
          "falseNegativeCount": "655",
          "trueNegativeCount": "493",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.89513993,
          "recall": 0.8299902,
          "precision": 0.9840487,
          "f1Score": 0.9004777,
          "falsePositiveRate": 0.09981851,
          "truePositiveCount": "3393",
          "falsePositiveCount": "55",
          "falseNegativeCount": "695",
          "trueNegativeCount": "496",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.90273213,
          "recall": 0.82020545,
          "precision": 0.9853071,
          "f1Score": 0.8952076,
          "falsePositiveRate": 0.0907441,
          "truePositiveCount": "3353",
          "falsePositiveCount": "50",
          "falseNegativeCount": "735",
          "trueNegativeCount": "501",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9118437,
          "recall": 0.81017613,
          "precision": 0.98718333,
          "f1Score": 0.88996375,
          "falsePositiveRate": 0.07803993,
          "truePositiveCount": "3312",
          "falsePositiveCount": "43",
          "falseNegativeCount": "776",
          "trueNegativeCount": "508",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9204901,
          "recall": 0.79990214,
          "precision": 0.98821396,
          "f1Score": 0.8841422,
          "falsePositiveRate": 0.0707804,
          "truePositiveCount": "3270",
          "falsePositiveCount": "39",
          "falseNegativeCount": "818",
          "trueNegativeCount": "512",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9283992,
          "recall": 0.7898728,
          "precision": 0.9898835,
          "f1Score": 0.87863946,
          "falsePositiveRate": 0.05989111,
          "truePositiveCount": "3229",
          "falsePositiveCount": "33",
          "falseNegativeCount": "859",
          "trueNegativeCount": "518",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.934707,
          "recall": 0.77984345,
          "precision": 0.9909854,
          "f1Score": 0.8728268,
          "falsePositiveRate": 0.05263158,
          "truePositiveCount": "3188",
          "falsePositiveCount": "29",
          "falseNegativeCount": "900",
          "trueNegativeCount": "522",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9401358,
          "recall": 0.7698141,
          "precision": 0.99211854,
          "f1Score": 0.86694217,
          "falsePositiveRate": 0.04537205,
          "truePositiveCount": "3147",
          "falsePositiveCount": "25",
          "falseNegativeCount": "941",
          "trueNegativeCount": "526",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.943966,
          "recall": 0.7600294,
          "precision": 0.9923347,
          "f1Score": 0.86078405,
          "falsePositiveRate": 0.043557167,
          "truePositiveCount": "3107",
          "falsePositiveCount": "24",
          "falseNegativeCount": "981",
          "trueNegativeCount": "527",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9484665,
          "recall": 0.7497554,
          "precision": 0.9925518,
          "f1Score": 0.85423636,
          "falsePositiveRate": 0.041742288,
          "truePositiveCount": "3065",
          "falsePositiveCount": "23",
          "falseNegativeCount": "1023",
          "trueNegativeCount": "528",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9512129,
          "recall": 0.7399706,
          "precision": 0.9931057,
          "f1Score": 0.8480516,
          "falsePositiveRate": 0.03811252,
          "truePositiveCount": "3025",
          "falsePositiveCount": "21",
          "falseNegativeCount": "1063",
          "trueNegativeCount": "530",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.95533496,
          "recall": 0.7299413,
          "precision": 0.9946667,
          "f1Score": 0.8419865,
          "falsePositiveRate": 0.029038113,
          "truePositiveCount": "2984",
          "falsePositiveCount": "16",
          "falseNegativeCount": "1104",
          "trueNegativeCount": "535",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.95724624,
          "recall": 0.72015655,
          "precision": 0.9956037,
          "f1Score": 0.8357701,
          "falsePositiveRate": 0.023593467,
          "truePositiveCount": "2944",
          "falsePositiveCount": "13",
          "falseNegativeCount": "1144",
          "trueNegativeCount": "538",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96008635,
          "recall": 0.7101272,
          "precision": 0.99588335,
          "f1Score": 0.82907325,
          "falsePositiveRate": 0.021778584,
          "truePositiveCount": "2903",
          "falsePositiveCount": "12",
          "falseNegativeCount": "1185",
          "trueNegativeCount": "539",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9629221,
          "recall": 0.70009786,
          "precision": 0.99617124,
          "f1Score": 0.82229567,
          "falsePositiveRate": 0.019963702,
          "truePositiveCount": "2862",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1226",
          "trueNegativeCount": "540",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9646139,
          "recall": 0.6900685,
          "precision": 0.9961158,
          "f1Score": 0.8153179,
          "falsePositiveRate": 0.019963702,
          "truePositiveCount": "2821",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1267",
          "trueNegativeCount": "540",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96656674,
          "recall": 0.67979455,
          "precision": 0.99605733,
          "f1Score": 0.8080838,
          "falsePositiveRate": 0.019963702,
          "truePositiveCount": "2779",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1309",
          "trueNegativeCount": "540",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96897614,
          "recall": 0.6700098,
          "precision": 0.9963623,
          "f1Score": 0.8012286,
          "falsePositiveRate": 0.01814882,
          "truePositiveCount": "2739",
          "falsePositiveCount": "10",
          "falseNegativeCount": "1349",
          "trueNegativeCount": "541",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9706901,
          "recall": 0.6599804,
          "precision": 0.99630725,
          "f1Score": 0.79399645,
          "falsePositiveRate": 0.01814882,
          "truePositiveCount": "2698",
          "falsePositiveCount": "10",
          "falseNegativeCount": "1390",
          "trueNegativeCount": "541",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.97229564,
          "recall": 0.6499511,
          "precision": 0.99699813,
          "f1Score": 0.7869095,
          "falsePositiveRate": 0.014519056,
          "truePositiveCount": "2657",
          "falsePositiveCount": "8",
          "falseNegativeCount": "1431",
          "trueNegativeCount": "543",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9738624,
          "recall": 0.6399217,
          "precision": 0.9969512,
          "f1Score": 0.7794994,
          "falsePositiveRate": 0.014519056,
          "truePositiveCount": "2616",
          "falsePositiveCount": "8",
          "falseNegativeCount": "1472",
          "trueNegativeCount": "543",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.97582275,
          "recall": 0.62989235,
          "precision": 0.99728894,
          "f1Score": 0.7721139,
          "falsePositiveRate": 0.012704174,
          "truePositiveCount": "2575",
          "falsePositiveCount": "7",
          "falseNegativeCount": "1513",
          "trueNegativeCount": "544",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9772092,
          "recall": 0.62010765,
          "precision": 0.9976387,
          "f1Score": 0.76482123,
          "falsePositiveRate": 0.010889292,
          "truePositiveCount": "2535",
          "falsePositiveCount": "6",
          "falseNegativeCount": "1553",
          "trueNegativeCount": "545",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9781838,
          "recall": 0.6100783,
          "precision": 0.9979992,
          "f1Score": 0.7572491,
          "falsePositiveRate": 0.00907441,
          "truePositiveCount": "2494",
          "falsePositiveCount": "5",
          "falseNegativeCount": "1594",
          "trueNegativeCount": "546",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9794321,
          "recall": 0.5998043,
          "precision": 0.99837136,
          "f1Score": 0.74938875,
          "falsePositiveRate": 0.007259528,
          "truePositiveCount": "2452",
          "falsePositiveCount": "4",
          "falseNegativeCount": "1636",
          "trueNegativeCount": "547",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9804444,
          "recall": 0.5900196,
          "precision": 0.99834436,
          "f1Score": 0.74169743,
          "falsePositiveRate": 0.007259528,
          "truePositiveCount": "2412",
          "falsePositiveCount": "4",
          "falseNegativeCount": "1676",
          "trueNegativeCount": "547",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9813577,
          "recall": 0.5799902,
          "precision": 0.9991572,
          "f1Score": 0.7339421,
          "falsePositiveRate": 0.003629764,
          "truePositiveCount": "2371",
          "falsePositiveCount": "2",
          "falseNegativeCount": "1717",
          "trueNegativeCount": "549",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9822546,
          "recall": 0.56996083,
          "precision": 0.99914235,
          "f1Score": 0.7258567,
          "falsePositiveRate": 0.003629764,
          "truePositiveCount": "2330",
          "falsePositiveCount": "2",
          "falseNegativeCount": "1758",
          "trueNegativeCount": "549",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9832565,
          "recall": 0.5599315,
          "precision": 0.99912703,
          "f1Score": 0.71766734,
          "falsePositiveRate": 0.003629764,
          "truePositiveCount": "2289",
          "falsePositiveCount": "2",
          "falseNegativeCount": "1799",
          "trueNegativeCount": "549",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.983971,
          "recall": 0.54990214,
          "precision": 0.99955535,
          "f1Score": 0.709484,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2248",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1840",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98478603,
          "recall": 0.54011744,
          "precision": 0.9995473,
          "f1Score": 0.7012863,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2208",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1880",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98576885,
          "recall": 0.52984345,
          "precision": 0.99953854,
          "f1Score": 0.6925659,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2166",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1922",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9865271,
          "recall": 0.5198141,
          "precision": 0.99952966,
          "f1Score": 0.6839395,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2125",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1963",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98722655,
          "recall": 0.5100294,
          "precision": 0.9995206,
          "f1Score": 0.675413,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2085",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2003",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98776835,
          "recall": 0.49975538,
          "precision": 0.99951077,
          "f1Score": 0.66634053,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2043",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2045",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98843795,
          "recall": 0.49021527,
          "precision": 0.9995012,
          "f1Score": 0.657804,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2004",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2084",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98898214,
          "recall": 0.4801859,
          "precision": 0.99949086,
          "f1Score": 0.64871114,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1963",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2125",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98952276,
          "recall": 0.46991193,
          "precision": 0.9994797,
          "f1Score": 0.63926786,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1921",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2167",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99007314,
          "recall": 0.4598826,
          "precision": 0.9994684,
          "f1Score": 0.62992126,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1880",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2208",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.990457,
          "recall": 0.45009786,
          "precision": 0.9994568,
          "f1Score": 0.620678,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1840",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2248",
          "trueNegativeCount": "550",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99079466,
          "recall": 0.44006848,
          "precision": 1.0,
          "f1Score": 0.61117715,
          "truePositiveCount": "1799",
          "falseNegativeCount": "2289",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9911339,
          "recall": 0.43003914,
          "precision": 1.0,
          "f1Score": 0.60143685,
          "truePositiveCount": "1758",
          "falseNegativeCount": "2330",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99141955,
          "recall": 0.4200098,
          "precision": 1.0,
          "f1Score": 0.591559,
          "truePositiveCount": "1717",
          "falseNegativeCount": "2371",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9917892,
          "recall": 0.40998042,
          "precision": 1.0,
          "f1Score": 0.5815406,
          "truePositiveCount": "1676",
          "falseNegativeCount": "2412",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9921363,
          "recall": 0.39995107,
          "precision": 1.0,
          "f1Score": 0.57137865,
          "truePositiveCount": "1635",
          "falseNegativeCount": "2453",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99253094,
          "recall": 0.38992172,
          "precision": 1.0,
          "f1Score": 0.56107,
          "truePositiveCount": "1594",
          "falseNegativeCount": "2494",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9928577,
          "recall": 0.380137,
          "precision": 1.0,
          "f1Score": 0.5508685,
          "truePositiveCount": "1554",
          "falseNegativeCount": "2534",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99313205,
          "recall": 0.369863,
          "precision": 1.0,
          "f1Score": 0.54,
          "truePositiveCount": "1512",
          "falseNegativeCount": "2576",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9933454,
          "recall": 0.35983366,
          "precision": 1.0,
          "f1Score": 0.5292319,
          "truePositiveCount": "1471",
          "falseNegativeCount": "2617",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99355274,
          "recall": 0.35004893,
          "precision": 1.0,
          "f1Score": 0.5185722,
          "truePositiveCount": "1431",
          "falseNegativeCount": "2657",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99378794,
          "recall": 0.33977494,
          "precision": 1.0,
          "f1Score": 0.507212,
          "truePositiveCount": "1389",
          "falseNegativeCount": "2699",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9939992,
          "recall": 0.33023483,
          "precision": 1.0,
          "f1Score": 0.49650607,
          "truePositiveCount": "1350",
          "falseNegativeCount": "2738",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9942213,
          "recall": 0.31996086,
          "precision": 1.0,
          "f1Score": 0.48480356,
          "truePositiveCount": "1308",
          "falseNegativeCount": "2780",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9944552,
          "recall": 0.30993152,
          "precision": 1.0,
          "f1Score": 0.47320262,
          "truePositiveCount": "1267",
          "falseNegativeCount": "2821",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9946339,
          "recall": 0.29990214,
          "precision": 1.0,
          "f1Score": 0.46142265,
          "truePositiveCount": "1226",
          "falseNegativeCount": "2862",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99486357,
          "recall": 0.2898728,
          "precision": 1.0,
          "f1Score": 0.44945952,
          "truePositiveCount": "1185",
          "falseNegativeCount": "2903",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9950511,
          "recall": 0.28008807,
          "precision": 1.0,
          "f1Score": 0.4376075,
          "truePositiveCount": "1145",
          "falseNegativeCount": "2943",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9953063,
          "recall": 0.2698141,
          "precision": 1.0,
          "f1Score": 0.42496628,
          "truePositiveCount": "1103",
          "falseNegativeCount": "2985",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99548787,
          "recall": 0.26002935,
          "precision": 1.0,
          "f1Score": 0.4127354,
          "truePositiveCount": "1063",
          "falseNegativeCount": "3025",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99564606,
          "recall": 0.25024462,
          "precision": 1.0,
          "f1Score": 0.40031305,
          "truePositiveCount": "1023",
          "falseNegativeCount": "3065",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9958494,
          "recall": 0.23997064,
          "precision": 1.0,
          "f1Score": 0.3870586,
          "truePositiveCount": "981",
          "falseNegativeCount": "3107",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9959897,
          "recall": 0.2299413,
          "precision": 1.0,
          "f1Score": 0.37390614,
          "truePositiveCount": "940",
          "falseNegativeCount": "3148",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9961461,
          "recall": 0.21991193,
          "precision": 1.0,
          "f1Score": 0.3605374,
          "truePositiveCount": "899",
          "falseNegativeCount": "3189",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99626255,
          "recall": 0.20988259,
          "precision": 1.0,
          "f1Score": 0.346947,
          "truePositiveCount": "858",
          "falseNegativeCount": "3230",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99635655,
          "recall": 0.19985323,
          "precision": 1.0,
          "f1Score": 0.33312947,
          "truePositiveCount": "817",
          "falseNegativeCount": "3271",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9964815,
          "recall": 0.1900685,
          "precision": 1.0,
          "f1Score": 0.31942445,
          "truePositiveCount": "777",
          "falseNegativeCount": "3311",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9966108,
          "recall": 0.17979452,
          "precision": 1.0,
          "f1Score": 0.30478954,
          "truePositiveCount": "735",
          "falseNegativeCount": "3353",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99673027,
          "recall": 0.16976516,
          "precision": 1.0,
          "f1Score": 0.29025513,
          "truePositiveCount": "694",
          "falseNegativeCount": "3394",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99681914,
          "recall": 0.15998043,
          "precision": 1.0,
          "f1Score": 0.27583298,
          "truePositiveCount": "654",
          "falseNegativeCount": "3434",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9969455,
          "recall": 0.15019569,
          "precision": 1.0,
          "f1Score": 0.26116547,
          "truePositiveCount": "614",
          "falseNegativeCount": "3474",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99705213,
          "recall": 0.13992172,
          "precision": 1.0,
          "f1Score": 0.24549356,
          "truePositiveCount": "572",
          "falseNegativeCount": "3516",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9971576,
          "recall": 0.13013698,
          "precision": 1.0,
          "f1Score": 0.23030303,
          "truePositiveCount": "532",
          "falseNegativeCount": "3556",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9972461,
          "recall": 0.11986301,
          "precision": 1.0,
          "f1Score": 0.21406728,
          "truePositiveCount": "490",
          "falseNegativeCount": "3598",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9973425,
          "recall": 0.110078275,
          "precision": 1.0,
          "f1Score": 0.19832525,
          "truePositiveCount": "450",
          "falseNegativeCount": "3638",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9974355,
          "recall": 0.10004892,
          "precision": 1.0,
          "f1Score": 0.18189904,
          "truePositiveCount": "409",
          "falseNegativeCount": "3679",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9975045,
          "recall": 0.09001957,
          "precision": 1.0,
          "f1Score": 0.16517055,
          "truePositiveCount": "368",
          "falseNegativeCount": "3720",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99760437,
          "recall": 0.079990216,
          "precision": 1.0,
          "f1Score": 0.14813137,
          "truePositiveCount": "327",
          "falseNegativeCount": "3761",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9976864,
          "recall": 0.07020548,
          "precision": 1.0,
          "f1Score": 0.1312,
          "truePositiveCount": "287",
          "falseNegativeCount": "3801",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9977548,
          "recall": 0.059931505,
          "precision": 1.0,
          "f1Score": 0.11308562,
          "truePositiveCount": "245",
          "falseNegativeCount": "3843",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99784184,
          "recall": 0.05014677,
          "precision": 1.0,
          "f1Score": 0.095504306,
          "truePositiveCount": "205",
          "falseNegativeCount": "3883",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9979226,
          "recall": 0.040117417,
          "precision": 1.0,
          "f1Score": 0.07714017,
          "truePositiveCount": "164",
          "falseNegativeCount": "3924",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9980229,
          "recall": 0.030088063,
          "precision": 1.0,
          "f1Score": 0.058418427,
          "truePositiveCount": "123",
          "falseNegativeCount": "3965",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9981275,
          "recall": 0.01981409,
          "precision": 1.0,
          "f1Score": 0.03885824,
          "truePositiveCount": "81",
          "falseNegativeCount": "4007",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9982374,
          "recall": 0.009784736,
          "precision": 1.0,
          "f1Score": 0.019379845,
          "truePositiveCount": "40",
          "falseNegativeCount": "4048",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9987167,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4088",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 1.0,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4088",
          "trueNegativeCount": "551",
          "positionThreshold": 2147483647
        }
      ],
      "auRoc": 0.9340445,
      "logLoss": 0.09388557
    },
    "displayName": "1"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/1660852072267559125",
    "annotationSpecId": "not available",
    "createTime": "2021-02-28T17:24:31.800758Z",
    "evaluatedExampleCount": 551,
    "classificationEvaluationMetrics": {
      "auPrc": 0.61527693,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.1187756,
          "f1Score": 0.2123314,
          "falsePositiveRate": 1.0,
          "truePositiveCount": "551",
          "falsePositiveCount": "4088",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.009267787,
          "recall": 0.9981851,
          "precision": 0.19400352,
          "f1Score": 0.3248671,
          "falsePositiveRate": 0.55895305,
          "truePositiveCount": "550",
          "falsePositiveCount": "2285",
          "falseNegativeCount": "1",
          "trueNegativeCount": "1803",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.022539802,
          "recall": 0.9891107,
          "precision": 0.2587844,
          "f1Score": 0.4102371,
          "falsePositiveRate": 0.38184932,
          "truePositiveCount": "545",
          "falsePositiveCount": "1561",
          "falseNegativeCount": "6",
          "trueNegativeCount": "2527",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.03954769,
          "recall": 0.9782214,
          "precision": 0.3113807,
          "f1Score": 0.47239265,
          "falsePositiveRate": 0.29158512,
          "truePositiveCount": "539",
          "falsePositiveCount": "1192",
          "falseNegativeCount": "12",
          "trueNegativeCount": "2896",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.045538254,
          "recall": 0.969147,
          "precision": 0.32902032,
          "f1Score": 0.49126035,
          "falsePositiveRate": 0.26638943,
          "truePositiveCount": "534",
          "falsePositiveCount": "1089",
          "falseNegativeCount": "17",
          "trueNegativeCount": "2999",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.050852664,
          "recall": 0.95825773,
          "precision": 0.33802816,
          "f1Score": 0.49976337,
          "falsePositiveRate": 0.2529354,
          "truePositiveCount": "528",
          "falsePositiveCount": "1034",
          "falseNegativeCount": "23",
          "trueNegativeCount": "3054",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.063637815,
          "recall": 0.9491833,
          "precision": 0.3647141,
          "f1Score": 0.52695215,
          "falsePositiveRate": 0.22284736,
          "truePositiveCount": "523",
          "falsePositiveCount": "911",
          "falseNegativeCount": "28",
          "trueNegativeCount": "3177",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.07210142,
          "recall": 0.938294,
          "precision": 0.377097,
          "f1Score": 0.5379813,
          "falsePositiveRate": 0.2089041,
          "truePositiveCount": "517",
          "falsePositiveCount": "854",
          "falseNegativeCount": "34",
          "trueNegativeCount": "3234",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.07750216,
          "recall": 0.9292196,
          "precision": 0.3826607,
          "f1Score": 0.54208577,
          "falsePositiveRate": 0.2020548,
          "truePositiveCount": "512",
          "falsePositiveCount": "826",
          "falseNegativeCount": "39",
          "trueNegativeCount": "3262",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.09185968,
          "recall": 0.9183303,
          "precision": 0.4,
          "f1Score": 0.55726874,
          "falsePositiveRate": 0.18566537,
          "truePositiveCount": "506",
          "falsePositiveCount": "759",
          "falseNegativeCount": "45",
          "trueNegativeCount": "3329",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.09674809,
          "recall": 0.9092559,
          "precision": 0.40501213,
          "f1Score": 0.5604027,
          "falsePositiveRate": 0.18003914,
          "truePositiveCount": "501",
          "falsePositiveCount": "736",
          "falseNegativeCount": "50",
          "trueNegativeCount": "3352",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.10816234,
          "recall": 0.89836663,
          "precision": 0.4212766,
          "f1Score": 0.5735805,
          "falsePositiveRate": 0.16634052,
          "truePositiveCount": "495",
          "falsePositiveCount": "680",
          "falseNegativeCount": "56",
          "trueNegativeCount": "3408",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.12236822,
          "recall": 0.8892922,
          "precision": 0.43672013,
          "f1Score": 0.58577406,
          "falsePositiveRate": 0.15459883,
          "truePositiveCount": "490",
          "falsePositiveCount": "632",
          "falseNegativeCount": "61",
          "trueNegativeCount": "3456",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.1267207,
          "recall": 0.8784029,
          "precision": 0.4368231,
          "f1Score": 0.58348405,
          "falsePositiveRate": 0.15264188,
          "truePositiveCount": "484",
          "falsePositiveCount": "624",
          "falseNegativeCount": "67",
          "trueNegativeCount": "3464",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.15273023,
          "recall": 0.8693285,
          "precision": 0.46550047,
          "f1Score": 0.60632914,
          "falsePositiveRate": 0.13454011,
          "truePositiveCount": "479",
          "falsePositiveCount": "550",
          "falseNegativeCount": "72",
          "trueNegativeCount": "3538",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.1649416,
          "recall": 0.8584392,
          "precision": 0.47252747,
          "f1Score": 0.6095361,
          "falsePositiveRate": 0.12915851,
          "truePositiveCount": "473",
          "falsePositiveCount": "528",
          "falseNegativeCount": "78",
          "trueNegativeCount": "3560",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.18867286,
          "recall": 0.8493648,
          "precision": 0.49628845,
          "f1Score": 0.62650603,
          "falsePositiveRate": 0.116193734,
          "truePositiveCount": "468",
          "falsePositiveCount": "475",
          "falseNegativeCount": "83",
          "trueNegativeCount": "3613",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.19824302,
          "recall": 0.8384755,
          "precision": 0.5038168,
          "f1Score": 0.6294278,
          "falsePositiveRate": 0.11130137,
          "truePositiveCount": "462",
          "falsePositiveCount": "455",
          "falseNegativeCount": "89",
          "trueNegativeCount": "3633",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.20855853,
          "recall": 0.8294011,
          "precision": 0.5163842,
          "f1Score": 0.6364902,
          "falsePositiveRate": 0.104696676,
          "truePositiveCount": "457",
          "falsePositiveCount": "428",
          "falseNegativeCount": "94",
          "trueNegativeCount": "3660",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.2164171,
          "recall": 0.8185118,
          "precision": 0.51898736,
          "f1Score": 0.6352113,
          "falsePositiveRate": 0.10225049,
          "truePositiveCount": "451",
          "falsePositiveCount": "418",
          "falseNegativeCount": "100",
          "trueNegativeCount": "3670",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.2228882,
          "recall": 0.8094374,
          "precision": 0.52347416,
          "f1Score": 0.63578045,
          "falsePositiveRate": 0.09931507,
          "truePositiveCount": "446",
          "falsePositiveCount": "406",
          "falseNegativeCount": "105",
          "trueNegativeCount": "3682",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.2343938,
          "recall": 0.7985481,
          "precision": 0.5339806,
          "f1Score": 0.64,
          "falsePositiveRate": 0.09393346,
          "truePositiveCount": "440",
          "falsePositiveCount": "384",
          "falseNegativeCount": "111",
          "trueNegativeCount": "3704",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.24437484,
          "recall": 0.7894737,
          "precision": 0.5390335,
          "f1Score": 0.640648,
          "falsePositiveRate": 0.090998046,
          "truePositiveCount": "435",
          "falsePositiveCount": "372",
          "falseNegativeCount": "116",
          "trueNegativeCount": "3716",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.24899487,
          "recall": 0.7785844,
          "precision": 0.53692114,
          "f1Score": 0.63555557,
          "falsePositiveRate": 0.0905088,
          "truePositiveCount": "429",
          "falsePositiveCount": "370",
          "falseNegativeCount": "122",
          "trueNegativeCount": "3718",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.25527748,
          "recall": 0.76951,
          "precision": 0.5401274,
          "f1Score": 0.6347305,
          "falsePositiveRate": 0.08830724,
          "truePositiveCount": "424",
          "falsePositiveCount": "361",
          "falseNegativeCount": "127",
          "trueNegativeCount": "3727",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.2643137,
          "recall": 0.7586207,
          "precision": 0.5414508,
          "f1Score": 0.6318972,
          "falsePositiveRate": 0.08659491,
          "truePositiveCount": "418",
          "falsePositiveCount": "354",
          "falseNegativeCount": "133",
          "trueNegativeCount": "3734",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.27015588,
          "recall": 0.7495463,
          "precision": 0.544137,
          "f1Score": 0.63053435,
          "falsePositiveRate": 0.08463796,
          "truePositiveCount": "413",
          "falsePositiveCount": "346",
          "falseNegativeCount": "138",
          "trueNegativeCount": "3742",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.2785622,
          "recall": 0.738657,
          "precision": 0.547043,
          "f1Score": 0.62857145,
          "falsePositiveRate": 0.0824364,
          "truePositiveCount": "407",
          "falsePositiveCount": "337",
          "falseNegativeCount": "144",
          "trueNegativeCount": "3751",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.29147997,
          "recall": 0.72958255,
          "precision": 0.5552486,
          "f1Score": 0.63058823,
          "falsePositiveRate": 0.07876712,
          "truePositiveCount": "402",
          "falsePositiveCount": "322",
          "falseNegativeCount": "149",
          "trueNegativeCount": "3766",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.30125153,
          "recall": 0.71869326,
          "precision": 0.55774647,
          "f1Score": 0.628073,
          "falsePositiveRate": 0.07681017,
          "truePositiveCount": "396",
          "falsePositiveCount": "314",
          "falseNegativeCount": "155",
          "trueNegativeCount": "3774",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.30796623,
          "recall": 0.70961887,
          "precision": 0.55857146,
          "f1Score": 0.6250999,
          "falsePositiveRate": 0.07558709,
          "truePositiveCount": "391",
          "falsePositiveCount": "309",
          "falseNegativeCount": "160",
          "trueNegativeCount": "3779",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.32498023,
          "recall": 0.6987296,
          "precision": 0.5636896,
          "f1Score": 0.623987,
          "falsePositiveRate": 0.07289628,
          "truePositiveCount": "385",
          "falsePositiveCount": "298",
          "falseNegativeCount": "166",
          "trueNegativeCount": "3790",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.33223236,
          "recall": 0.6896552,
          "precision": 0.5654762,
          "f1Score": 0.6214227,
          "falsePositiveRate": 0.071428575,
          "truePositiveCount": "380",
          "falsePositiveCount": "292",
          "falseNegativeCount": "171",
          "trueNegativeCount": "3796",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.3348615,
          "recall": 0.6787659,
          "precision": 0.562406,
          "f1Score": 0.61513156,
          "falsePositiveRate": 0.07118395,
          "truePositiveCount": "374",
          "falsePositiveCount": "291",
          "falseNegativeCount": "177",
          "trueNegativeCount": "3797",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.33746937,
          "recall": 0.66969144,
          "precision": 0.5607903,
          "f1Score": 0.61042184,
          "falsePositiveRate": 0.070694715,
          "truePositiveCount": "369",
          "falsePositiveCount": "289",
          "falseNegativeCount": "182",
          "trueNegativeCount": "3799",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.34359238,
          "recall": 0.65880215,
          "precision": 0.561051,
          "f1Score": 0.60601,
          "falsePositiveRate": 0.06947163,
          "truePositiveCount": "363",
          "falsePositiveCount": "284",
          "falseNegativeCount": "188",
          "trueNegativeCount": "3804",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.3495217,
          "recall": 0.64972776,
          "precision": 0.5585023,
          "f1Score": 0.6006711,
          "falsePositiveRate": 0.069227,
          "truePositiveCount": "358",
          "falsePositiveCount": "283",
          "falseNegativeCount": "193",
          "trueNegativeCount": "3805",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.35605764,
          "recall": 0.63883847,
          "precision": 0.5623003,
          "f1Score": 0.5981308,
          "falsePositiveRate": 0.06702544,
          "truePositiveCount": "352",
          "falsePositiveCount": "274",
          "falseNegativeCount": "199",
          "trueNegativeCount": "3814",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.36666742,
          "recall": 0.6297641,
          "precision": 0.56978655,
          "f1Score": 0.59827584,
          "falsePositiveRate": 0.06409002,
          "truePositiveCount": "347",
          "falsePositiveCount": "262",
          "falseNegativeCount": "204",
          "trueNegativeCount": "3826",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.37787727,
          "recall": 0.6188748,
          "precision": 0.5809199,
          "f1Score": 0.599297,
          "falsePositiveRate": 0.060176127,
          "truePositiveCount": "341",
          "falsePositiveCount": "246",
          "falseNegativeCount": "210",
          "trueNegativeCount": "3842",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.39064476,
          "recall": 0.60980034,
          "precision": 0.58638746,
          "f1Score": 0.59786475,
          "falsePositiveRate": 0.05797456,
          "truePositiveCount": "336",
          "falsePositiveCount": "237",
          "falseNegativeCount": "215",
          "trueNegativeCount": "3851",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.39657614,
          "recall": 0.59891105,
          "precision": 0.5871886,
          "f1Score": 0.5929919,
          "falsePositiveRate": 0.056751467,
          "truePositiveCount": "330",
          "falsePositiveCount": "232",
          "falseNegativeCount": "221",
          "trueNegativeCount": "3856",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.39991632,
          "recall": 0.58983666,
          "precision": 0.5866426,
          "f1Score": 0.5882353,
          "falsePositiveRate": 0.05601761,
          "truePositiveCount": "325",
          "falsePositiveCount": "229",
          "falseNegativeCount": "226",
          "trueNegativeCount": "3859",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.41141164,
          "recall": 0.57894737,
          "precision": 0.5896488,
          "f1Score": 0.5842491,
          "falsePositiveRate": 0.054305285,
          "truePositiveCount": "319",
          "falsePositiveCount": "222",
          "falseNegativeCount": "232",
          "trueNegativeCount": "3866",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.4184669,
          "recall": 0.569873,
          "precision": 0.5902256,
          "f1Score": 0.5798707,
          "falsePositiveRate": 0.05332681,
          "truePositiveCount": "314",
          "falsePositiveCount": "218",
          "falseNegativeCount": "237",
          "trueNegativeCount": "3870",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.42953506,
          "recall": 0.5589837,
          "precision": 0.5980582,
          "f1Score": 0.5778612,
          "falsePositiveRate": 0.05063601,
          "truePositiveCount": "308",
          "falsePositiveCount": "207",
          "falseNegativeCount": "243",
          "trueNegativeCount": "3881",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.43759817,
          "recall": 0.54990923,
          "precision": 0.6023857,
          "f1Score": 0.57495254,
          "falsePositiveRate": 0.04892368,
          "truePositiveCount": "303",
          "falsePositiveCount": "200",
          "falseNegativeCount": "248",
          "trueNegativeCount": "3888",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.44597256,
          "recall": 0.53901994,
          "precision": 0.60365856,
          "f1Score": 0.569511,
          "falsePositiveRate": 0.047700588,
          "truePositiveCount": "297",
          "falsePositiveCount": "195",
          "falseNegativeCount": "254",
          "trueNegativeCount": "3893",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.44895133,
          "recall": 0.52994555,
          "precision": 0.6045549,
          "f1Score": 0.5647969,
          "falsePositiveRate": 0.046722114,
          "truePositiveCount": "292",
          "falsePositiveCount": "191",
          "falseNegativeCount": "259",
          "trueNegativeCount": "3897",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.46139872,
          "recall": 0.51905626,
          "precision": 0.6085106,
          "f1Score": 0.5602351,
          "falsePositiveRate": 0.045009784,
          "truePositiveCount": "286",
          "falsePositiveCount": "184",
          "falseNegativeCount": "265",
          "trueNegativeCount": "3904",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.47061405,
          "recall": 0.5099819,
          "precision": 0.61622804,
          "f1Score": 0.55809337,
          "falsePositiveRate": 0.04280822,
          "truePositiveCount": "281",
          "falsePositiveCount": "175",
          "falseNegativeCount": "270",
          "trueNegativeCount": "3913",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.48159525,
          "recall": 0.49909255,
          "precision": 0.6207675,
          "f1Score": 0.55331993,
          "falsePositiveRate": 0.04109589,
          "truePositiveCount": "275",
          "falsePositiveCount": "168",
          "falseNegativeCount": "276",
          "trueNegativeCount": "3920",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.48736593,
          "recall": 0.48820326,
          "precision": 0.6226852,
          "f1Score": 0.54730415,
          "falsePositiveRate": 0.0398728,
          "truePositiveCount": "269",
          "falsePositiveCount": "163",
          "falseNegativeCount": "282",
          "trueNegativeCount": "3925",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.4939781,
          "recall": 0.47912887,
          "precision": 0.6300716,
          "f1Score": 0.5443299,
          "falsePositiveRate": 0.037915852,
          "truePositiveCount": "264",
          "falsePositiveCount": "155",
          "falseNegativeCount": "287",
          "trueNegativeCount": "3933",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.4995014,
          "recall": 0.46823958,
          "precision": 0.635468,
          "f1Score": 0.5391849,
          "falsePositiveRate": 0.036203522,
          "truePositiveCount": "258",
          "falsePositiveCount": "148",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3940",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5025096,
          "recall": 0.45916516,
          "precision": 0.63408524,
          "f1Score": 0.5326316,
          "falsePositiveRate": 0.035714287,
          "truePositiveCount": "253",
          "falsePositiveCount": "146",
          "falseNegativeCount": "298",
          "trueNegativeCount": "3942",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.51137525,
          "recall": 0.44827586,
          "precision": 0.63010204,
          "f1Score": 0.52386004,
          "falsePositiveRate": 0.035469666,
          "truePositiveCount": "247",
          "falsePositiveCount": "145",
          "falseNegativeCount": "304",
          "trueNegativeCount": "3943",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.51734245,
          "recall": 0.43920144,
          "precision": 0.62857145,
          "f1Score": 0.517094,
          "falsePositiveRate": 0.03498043,
          "truePositiveCount": "242",
          "falsePositiveCount": "143",
          "falseNegativeCount": "309",
          "trueNegativeCount": "3945",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5248498,
          "recall": 0.42831215,
          "precision": 0.631016,
          "f1Score": 0.5102703,
          "falsePositiveRate": 0.03375734,
          "truePositiveCount": "236",
          "falsePositiveCount": "138",
          "falseNegativeCount": "315",
          "trueNegativeCount": "3950",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.53241414,
          "recall": 0.41923776,
          "precision": 0.64705884,
          "f1Score": 0.5088106,
          "falsePositiveRate": 0.030821918,
          "truePositiveCount": "231",
          "falsePositiveCount": "126",
          "falseNegativeCount": "320",
          "trueNegativeCount": "3962",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5427009,
          "recall": 0.40834847,
          "precision": 0.64841497,
          "f1Score": 0.5011136,
          "falsePositiveRate": 0.029843444,
          "truePositiveCount": "225",
          "falsePositiveCount": "122",
          "falseNegativeCount": "326",
          "trueNegativeCount": "3966",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5505563,
          "recall": 0.39927405,
          "precision": 0.6547619,
          "f1Score": 0.4960541,
          "falsePositiveRate": 0.028375734,
          "truePositiveCount": "220",
          "falsePositiveCount": "116",
          "falseNegativeCount": "331",
          "trueNegativeCount": "3972",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.55334836,
          "recall": 0.38838476,
          "precision": 0.65045595,
          "f1Score": 0.48636365,
          "falsePositiveRate": 0.028131116,
          "truePositiveCount": "214",
          "falsePositiveCount": "115",
          "falseNegativeCount": "337",
          "trueNegativeCount": "3973",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.55837405,
          "recall": 0.37931034,
          "precision": 0.6490683,
          "f1Score": 0.4788087,
          "falsePositiveRate": 0.02764188,
          "truePositiveCount": "209",
          "falsePositiveCount": "113",
          "falseNegativeCount": "342",
          "trueNegativeCount": "3975",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5685674,
          "recall": 0.36842105,
          "precision": 0.650641,
          "f1Score": 0.47045192,
          "falsePositiveRate": 0.026663406,
          "truePositiveCount": "203",
          "falsePositiveCount": "109",
          "falseNegativeCount": "348",
          "trueNegativeCount": "3979",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5732897,
          "recall": 0.35934663,
          "precision": 0.6578073,
          "f1Score": 0.46478873,
          "falsePositiveRate": 0.025195695,
          "truePositiveCount": "198",
          "falsePositiveCount": "103",
          "falseNegativeCount": "353",
          "trueNegativeCount": "3985",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5798386,
          "recall": 0.34845734,
          "precision": 0.65753424,
          "f1Score": 0.455516,
          "falsePositiveRate": 0.02446184,
          "truePositiveCount": "192",
          "falsePositiveCount": "100",
          "falseNegativeCount": "359",
          "trueNegativeCount": "3988",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5841479,
          "recall": 0.33938295,
          "precision": 0.6561403,
          "f1Score": 0.4473684,
          "falsePositiveRate": 0.023972603,
          "truePositiveCount": "187",
          "falsePositiveCount": "98",
          "falseNegativeCount": "364",
          "trueNegativeCount": "3990",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.58893955,
          "recall": 0.32849365,
          "precision": 0.6630037,
          "f1Score": 0.4393204,
          "falsePositiveRate": 0.022504892,
          "truePositiveCount": "181",
          "falsePositiveCount": "92",
          "falseNegativeCount": "370",
          "trueNegativeCount": "3996",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.59818196,
          "recall": 0.31941923,
          "precision": 0.6743295,
          "f1Score": 0.43349755,
          "falsePositiveRate": 0.020792564,
          "truePositiveCount": "176",
          "falsePositiveCount": "85",
          "falseNegativeCount": "375",
          "trueNegativeCount": "4003",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.60704446,
          "recall": 0.30852994,
          "precision": 0.6854839,
          "f1Score": 0.42553192,
          "falsePositiveRate": 0.019080235,
          "truePositiveCount": "170",
          "falsePositiveCount": "78",
          "falseNegativeCount": "381",
          "trueNegativeCount": "4010",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6096991,
          "recall": 0.29945552,
          "precision": 0.6818182,
          "f1Score": 0.41614124,
          "falsePositiveRate": 0.018835617,
          "truePositiveCount": "165",
          "falsePositiveCount": "77",
          "falseNegativeCount": "386",
          "trueNegativeCount": "4011",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6194289,
          "recall": 0.28856623,
          "precision": 0.6913043,
          "f1Score": 0.4071703,
          "falsePositiveRate": 0.017367907,
          "truePositiveCount": "159",
          "falsePositiveCount": "71",
          "falseNegativeCount": "392",
          "trueNegativeCount": "4017",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6299003,
          "recall": 0.27949184,
          "precision": 0.6936937,
          "f1Score": 0.3984476,
          "falsePositiveRate": 0.01663405,
          "truePositiveCount": "154",
          "falsePositiveCount": "68",
          "falseNegativeCount": "397",
          "trueNegativeCount": "4020",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6326805,
          "recall": 0.26860255,
          "precision": 0.6883721,
          "f1Score": 0.38642296,
          "falsePositiveRate": 0.016389433,
          "truePositiveCount": "148",
          "falsePositiveCount": "67",
          "falseNegativeCount": "403",
          "trueNegativeCount": "4021",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6446329,
          "recall": 0.25952813,
          "precision": 0.71859294,
          "f1Score": 0.38133332,
          "falsePositiveRate": 0.01369863,
          "truePositiveCount": "143",
          "falsePositiveCount": "56",
          "falseNegativeCount": "408",
          "trueNegativeCount": "4032",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6525863,
          "recall": 0.24863884,
          "precision": 0.7287234,
          "f1Score": 0.37077132,
          "falsePositiveRate": 0.012475538,
          "truePositiveCount": "137",
          "falsePositiveCount": "51",
          "falseNegativeCount": "414",
          "trueNegativeCount": "4037",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6613302,
          "recall": 0.23956443,
          "precision": 0.73333335,
          "f1Score": 0.3611491,
          "falsePositiveRate": 0.011741683,
          "truePositiveCount": "132",
          "falsePositiveCount": "48",
          "falseNegativeCount": "419",
          "trueNegativeCount": "4040",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6681088,
          "recall": 0.22867514,
          "precision": 0.73255813,
          "f1Score": 0.34854773,
          "falsePositiveRate": 0.011252446,
          "truePositiveCount": "126",
          "falsePositiveCount": "46",
          "falseNegativeCount": "425",
          "trueNegativeCount": "4042",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6787454,
          "recall": 0.21960072,
          "precision": 0.7378049,
          "f1Score": 0.33846155,
          "falsePositiveRate": 0.010518591,
          "truePositiveCount": "121",
          "falsePositiveCount": "43",
          "falseNegativeCount": "430",
          "trueNegativeCount": "4045",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.68742466,
          "recall": 0.20871143,
          "precision": 0.73717946,
          "f1Score": 0.32531825,
          "falsePositiveRate": 0.010029354,
          "truePositiveCount": "115",
          "falsePositiveCount": "41",
          "falseNegativeCount": "436",
          "trueNegativeCount": "4047",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.69688004,
          "recall": 0.19963703,
          "precision": 0.7482993,
          "f1Score": 0.31518623,
          "falsePositiveRate": 0.009050881,
          "truePositiveCount": "110",
          "falsePositiveCount": "37",
          "falseNegativeCount": "441",
          "trueNegativeCount": "4051",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.70484525,
          "recall": 0.18874773,
          "precision": 0.7647059,
          "f1Score": 0.30276564,
          "falsePositiveRate": 0.007827789,
          "truePositiveCount": "104",
          "falsePositiveCount": "32",
          "falseNegativeCount": "447",
          "trueNegativeCount": "4056",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.71349317,
          "recall": 0.17967331,
          "precision": 0.76153845,
          "f1Score": 0.2907489,
          "falsePositiveRate": 0.00758317,
          "truePositiveCount": "99",
          "falsePositiveCount": "31",
          "falseNegativeCount": "452",
          "trueNegativeCount": "4057",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.72145003,
          "recall": 0.16878402,
          "precision": 0.75609756,
          "f1Score": 0.27596438,
          "falsePositiveRate": 0.007338552,
          "truePositiveCount": "93",
          "falsePositiveCount": "30",
          "falseNegativeCount": "458",
          "trueNegativeCount": "4058",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.72871804,
          "recall": 0.15970962,
          "precision": 0.76521736,
          "f1Score": 0.26426426,
          "falsePositiveRate": 0.0066046966,
          "truePositiveCount": "88",
          "falsePositiveCount": "27",
          "falseNegativeCount": "463",
          "trueNegativeCount": "4061",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.74161375,
          "recall": 0.14882033,
          "precision": 0.76635516,
          "f1Score": 0.24924012,
          "falsePositiveRate": 0.00611546,
          "truePositiveCount": "82",
          "falsePositiveCount": "25",
          "falseNegativeCount": "469",
          "trueNegativeCount": "4063",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.74766874,
          "recall": 0.13974592,
          "precision": 0.78571427,
          "f1Score": 0.23728813,
          "falsePositiveRate": 0.0051369863,
          "truePositiveCount": "77",
          "falsePositiveCount": "21",
          "falseNegativeCount": "474",
          "trueNegativeCount": "4067",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.75845456,
          "recall": 0.12885663,
          "precision": 0.8068182,
          "f1Score": 0.22222222,
          "falsePositiveRate": 0.0041585127,
          "truePositiveCount": "71",
          "falsePositiveCount": "17",
          "falseNegativeCount": "480",
          "trueNegativeCount": "4071",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7666716,
          "recall": 0.11978222,
          "precision": 0.80487806,
          "f1Score": 0.2085308,
          "falsePositiveRate": 0.0039138943,
          "truePositiveCount": "66",
          "falsePositiveCount": "16",
          "falseNegativeCount": "485",
          "trueNegativeCount": "4072",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7772814,
          "recall": 0.108892925,
          "precision": 0.7894737,
          "f1Score": 0.19138756,
          "falsePositiveRate": 0.0039138943,
          "truePositiveCount": "60",
          "falsePositiveCount": "16",
          "falseNegativeCount": "491",
          "trueNegativeCount": "4072",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.78661436,
          "recall": 0.09981851,
          "precision": 0.8088235,
          "f1Score": 0.17770597,
          "falsePositiveRate": 0.003180039,
          "truePositiveCount": "55",
          "falsePositiveCount": "13",
          "falseNegativeCount": "496",
          "trueNegativeCount": "4075",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.8050728,
          "recall": 0.08892922,
          "precision": 0.8448276,
          "f1Score": 0.16091955,
          "falsePositiveRate": 0.0022015655,
          "truePositiveCount": "49",
          "falsePositiveCount": "9",
          "falseNegativeCount": "502",
          "trueNegativeCount": "4079",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.8127946,
          "recall": 0.07985481,
          "precision": 0.8301887,
          "f1Score": 0.14569536,
          "falsePositiveRate": 0.0022015655,
          "truePositiveCount": "44",
          "falsePositiveCount": "9",
          "falseNegativeCount": "507",
          "trueNegativeCount": "4079",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.82794523,
          "recall": 0.06896552,
          "precision": 0.82608694,
          "f1Score": 0.12730318,
          "falsePositiveRate": 0.0019569471,
          "truePositiveCount": "38",
          "falsePositiveCount": "8",
          "falseNegativeCount": "513",
          "trueNegativeCount": "4080",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.83858025,
          "recall": 0.05989111,
          "precision": 0.84615386,
          "f1Score": 0.11186441,
          "falsePositiveRate": 0.0014677104,
          "truePositiveCount": "33",
          "falsePositiveCount": "6",
          "falseNegativeCount": "518",
          "trueNegativeCount": "4082",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.85226774,
          "recall": 0.049001817,
          "precision": 0.87096775,
          "f1Score": 0.0927835,
          "falsePositiveRate": 0.0009784736,
          "truePositiveCount": "27",
          "falsePositiveCount": "4",
          "falseNegativeCount": "524",
          "trueNegativeCount": "4084",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.86955357,
          "recall": 0.039927404,
          "precision": 0.88,
          "f1Score": 0.07638889,
          "falsePositiveRate": 0.0007338552,
          "truePositiveCount": "22",
          "falsePositiveCount": "3",
          "falseNegativeCount": "529",
          "trueNegativeCount": "4085",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.8754055,
          "recall": 0.029038113,
          "precision": 0.84210527,
          "f1Score": 0.056140352,
          "falsePositiveRate": 0.0007338552,
          "truePositiveCount": "16",
          "falsePositiveCount": "3",
          "falseNegativeCount": "535",
          "trueNegativeCount": "4085",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.888999,
          "recall": 0.019963702,
          "precision": 0.78571427,
          "f1Score": 0.038938053,
          "falsePositiveRate": 0.0007338552,
          "truePositiveCount": "11",
          "falsePositiveCount": "3",
          "falseNegativeCount": "540",
          "trueNegativeCount": "4085",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.912919,
          "recall": 0.00907441,
          "precision": 0.71428573,
          "f1Score": 0.017921148,
          "falsePositiveRate": 0.0004892368,
          "truePositiveCount": "5",
          "falsePositiveCount": "2",
          "falseNegativeCount": "546",
          "trueNegativeCount": "4086",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9486238,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "551",
          "trueNegativeCount": "4088",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 1.0,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "551",
          "trueNegativeCount": "4088",
          "positionThreshold": 2147483647
        }
      ],
      "auRoc": 0.9330749,
      "logLoss": 0.993795
    },
    "displayName": "2"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/5840134723993193762",
    "annotationSpecId": "not available",
    "createTime": "2021-02-28T17:24:31.800758Z",
    "evaluatedExampleCount": 4088,
    "classificationEvaluationMetrics": {
      "auPrc": 0.9580479,
      "confidenceMetricsEntry": [
        {
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.07797722,
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.30998033,
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.39478266,
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.46149504,
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.9640411,
          "precision": 0.9307983,
          "f1Score": 0.9471281,
          "falsePositiveRate": 0.53176045,
          "truePositiveCount": "3941",
          "falsePositiveCount": "293",
          "falseNegativeCount": "147",
          "trueNegativeCount": "258",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.51386535,
          "recall": 0.95988256,
          "precision": 0.93361884,
          "f1Score": 0.94656855,
          "falsePositiveRate": 0.50635207,
          "truePositiveCount": "3924",
          "falsePositiveCount": "279",
          "falseNegativeCount": "164",
          "trueNegativeCount": "272",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5659395,
          "recall": 0.94985324,
          "precision": 0.9406492,
          "f1Score": 0.9452288,
          "falsePositiveRate": 0.4446461,
          "truePositiveCount": "3883",
          "falsePositiveCount": "245",
          "falseNegativeCount": "205",
          "trueNegativeCount": "306",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6215324,
          "recall": 0.93982387,
          "precision": 0.94817376,
          "f1Score": 0.94398034,
          "falsePositiveRate": 0.38112524,
          "truePositiveCount": "3842",
          "falsePositiveCount": "210",
          "falseNegativeCount": "246",
          "trueNegativeCount": "341",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.65884733,
          "recall": 0.93003917,
          "precision": 0.9533601,
          "f1Score": 0.9415552,
          "falsePositiveRate": 0.33756804,
          "truePositiveCount": "3802",
          "falsePositiveCount": "186",
          "falseNegativeCount": "286",
          "trueNegativeCount": "365",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7143411,
          "recall": 0.9200098,
          "precision": 0.9623849,
          "f1Score": 0.9407204,
          "falsePositiveRate": 0.26678765,
          "truePositiveCount": "3761",
          "falsePositiveCount": "147",
          "falseNegativeCount": "327",
          "trueNegativeCount": "404",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7492718,
          "recall": 0.9099804,
          "precision": 0.96749026,
          "f1Score": 0.9378545,
          "falsePositiveRate": 0.22686026,
          "truePositiveCount": "3720",
          "falsePositiveCount": "125",
          "falseNegativeCount": "368",
          "trueNegativeCount": "426",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7795446,
          "recall": 0.9001957,
          "precision": 0.97303015,
          "f1Score": 0.93519694,
          "falsePositiveRate": 0.18511796,
          "truePositiveCount": "3680",
          "falsePositiveCount": "102",
          "falseNegativeCount": "408",
          "trueNegativeCount": "449",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7989856,
          "recall": 0.8899217,
          "precision": 0.97559667,
          "f1Score": 0.93079185,
          "falsePositiveRate": 0.16515426,
          "truePositiveCount": "3638",
          "falsePositiveCount": "91",
          "falseNegativeCount": "450",
          "trueNegativeCount": "460",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.82042044,
          "recall": 0.87989235,
          "precision": 0.9777113,
          "f1Score": 0.9262263,
          "falsePositiveRate": 0.14882033,
          "truePositiveCount": "3597",
          "falsePositiveCount": "82",
          "falseNegativeCount": "491",
          "trueNegativeCount": "469",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.83562684,
          "recall": 0.87010765,
          "precision": 0.9790807,
          "f1Score": 0.92138326,
          "falsePositiveRate": 0.13793103,
          "truePositiveCount": "3557",
          "falsePositiveCount": "76",
          "falseNegativeCount": "531",
          "trueNegativeCount": "475",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.85593396,
          "recall": 0.8600783,
          "precision": 0.98102677,
          "f1Score": 0.9165798,
          "falsePositiveRate": 0.123411976,
          "truePositiveCount": "3516",
          "falsePositiveCount": "68",
          "falseNegativeCount": "572",
          "trueNegativeCount": "483",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.86959773,
          "recall": 0.8500489,
          "precision": 0.9810841,
          "f1Score": 0.9108781,
          "falsePositiveRate": 0.1215971,
          "truePositiveCount": "3475",
          "falsePositiveCount": "67",
          "falseNegativeCount": "613",
          "trueNegativeCount": "484",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.88389677,
          "recall": 0.83977497,
          "precision": 0.98338586,
          "f1Score": 0.90592426,
          "falsePositiveRate": 0.10526316,
          "truePositiveCount": "3433",
          "falsePositiveCount": "58",
          "falseNegativeCount": "655",
          "trueNegativeCount": "493",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.89513993,
          "recall": 0.8299902,
          "precision": 0.9840487,
          "f1Score": 0.9004777,
          "falsePositiveRate": 0.09981851,
          "truePositiveCount": "3393",
          "falsePositiveCount": "55",
          "falseNegativeCount": "695",
          "trueNegativeCount": "496",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.90273213,
          "recall": 0.82020545,
          "precision": 0.9853071,
          "f1Score": 0.8952076,
          "falsePositiveRate": 0.0907441,
          "truePositiveCount": "3353",
          "falsePositiveCount": "50",
          "falseNegativeCount": "735",
          "trueNegativeCount": "501",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9118437,
          "recall": 0.81017613,
          "precision": 0.98718333,
          "f1Score": 0.88996375,
          "falsePositiveRate": 0.07803993,
          "truePositiveCount": "3312",
          "falsePositiveCount": "43",
          "falseNegativeCount": "776",
          "trueNegativeCount": "508",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9204901,
          "recall": 0.79990214,
          "precision": 0.98821396,
          "f1Score": 0.8841422,
          "falsePositiveRate": 0.0707804,
          "truePositiveCount": "3270",
          "falsePositiveCount": "39",
          "falseNegativeCount": "818",
          "trueNegativeCount": "512",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9283992,
          "recall": 0.7898728,
          "precision": 0.9898835,
          "f1Score": 0.87863946,
          "falsePositiveRate": 0.05989111,
          "truePositiveCount": "3229",
          "falsePositiveCount": "33",
          "falseNegativeCount": "859",
          "trueNegativeCount": "518",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.934707,
          "recall": 0.77984345,
          "precision": 0.9909854,
          "f1Score": 0.8728268,
          "falsePositiveRate": 0.05263158,
          "truePositiveCount": "3188",
          "falsePositiveCount": "29",
          "falseNegativeCount": "900",
          "trueNegativeCount": "522",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9401358,
          "recall": 0.7698141,
          "precision": 0.99211854,
          "f1Score": 0.86694217,
          "falsePositiveRate": 0.04537205,
          "truePositiveCount": "3147",
          "falsePositiveCount": "25",
          "falseNegativeCount": "941",
          "trueNegativeCount": "526",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.943966,
          "recall": 0.7600294,
          "precision": 0.9923347,
          "f1Score": 0.86078405,
          "falsePositiveRate": 0.043557167,
          "truePositiveCount": "3107",
          "falsePositiveCount": "24",
          "falseNegativeCount": "981",
          "trueNegativeCount": "527",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9484665,
          "recall": 0.7497554,
          "precision": 0.9925518,
          "f1Score": 0.85423636,
          "falsePositiveRate": 0.041742288,
          "truePositiveCount": "3065",
          "falsePositiveCount": "23",
          "falseNegativeCount": "1023",
          "trueNegativeCount": "528",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9512129,
          "recall": 0.7399706,
          "precision": 0.9931057,
          "f1Score": 0.8480516,
          "falsePositiveRate": 0.03811252,
          "truePositiveCount": "3025",
          "falsePositiveCount": "21",
          "falseNegativeCount": "1063",
          "trueNegativeCount": "530",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.95533496,
          "recall": 0.7299413,
          "precision": 0.9946667,
          "f1Score": 0.8419865,
          "falsePositiveRate": 0.029038113,
          "truePositiveCount": "2984",
          "falsePositiveCount": "16",
          "falseNegativeCount": "1104",
          "trueNegativeCount": "535",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.95724624,
          "recall": 0.72015655,
          "precision": 0.9956037,
          "f1Score": 0.8357701,
          "falsePositiveRate": 0.023593467,
          "truePositiveCount": "2944",
          "falsePositiveCount": "13",
          "falseNegativeCount": "1144",
          "trueNegativeCount": "538",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96008635,
          "recall": 0.7101272,
          "precision": 0.99588335,
          "f1Score": 0.82907325,
          "falsePositiveRate": 0.021778584,
          "truePositiveCount": "2903",
          "falsePositiveCount": "12",
          "falseNegativeCount": "1185",
          "trueNegativeCount": "539",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9629221,
          "recall": 0.70009786,
          "precision": 0.99617124,
          "f1Score": 0.82229567,
          "falsePositiveRate": 0.019963702,
          "truePositiveCount": "2862",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1226",
          "trueNegativeCount": "540",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9646139,
          "recall": 0.6900685,
          "precision": 0.9961158,
          "f1Score": 0.8153179,
          "falsePositiveRate": 0.019963702,
          "truePositiveCount": "2821",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1267",
          "trueNegativeCount": "540",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96656674,
          "recall": 0.67979455,
          "precision": 0.99605733,
          "f1Score": 0.8080838,
          "falsePositiveRate": 0.019963702,
          "truePositiveCount": "2779",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1309",
          "trueNegativeCount": "540",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.96897614,
          "recall": 0.6700098,
          "precision": 0.9963623,
          "f1Score": 0.8012286,
          "falsePositiveRate": 0.01814882,
          "truePositiveCount": "2739",
          "falsePositiveCount": "10",
          "falseNegativeCount": "1349",
          "trueNegativeCount": "541",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9706901,
          "recall": 0.6599804,
          "precision": 0.99630725,
          "f1Score": 0.79399645,
          "falsePositiveRate": 0.01814882,
          "truePositiveCount": "2698",
          "falsePositiveCount": "10",
          "falseNegativeCount": "1390",
          "trueNegativeCount": "541",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.97229564,
          "recall": 0.6499511,
          "precision": 0.99699813,
          "f1Score": 0.7869095,
          "falsePositiveRate": 0.014519056,
          "truePositiveCount": "2657",
          "falsePositiveCount": "8",
          "falseNegativeCount": "1431",
          "trueNegativeCount": "543",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9738624,
          "recall": 0.6399217,
          "precision": 0.9969512,
          "f1Score": 0.7794994,
          "falsePositiveRate": 0.014519056,
          "truePositiveCount": "2616",
          "falsePositiveCount": "8",
          "falseNegativeCount": "1472",
          "trueNegativeCount": "543",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.97582275,
          "recall": 0.62989235,
          "precision": 0.99728894,
          "f1Score": 0.7721139,
          "falsePositiveRate": 0.012704174,
          "truePositiveCount": "2575",
          "falsePositiveCount": "7",
          "falseNegativeCount": "1513",
          "trueNegativeCount": "544",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9772092,
          "recall": 0.62010765,
          "precision": 0.9976387,
          "f1Score": 0.76482123,
          "falsePositiveRate": 0.010889292,
          "truePositiveCount": "2535",
          "falsePositiveCount": "6",
          "falseNegativeCount": "1553",
          "trueNegativeCount": "545",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9781838,
          "recall": 0.6100783,
          "precision": 0.9979992,
          "f1Score": 0.7572491,
          "falsePositiveRate": 0.00907441,
          "truePositiveCount": "2494",
          "falsePositiveCount": "5",
          "falseNegativeCount": "1594",
          "trueNegativeCount": "546",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9794321,
          "recall": 0.5998043,
          "precision": 0.99837136,
          "f1Score": 0.74938875,
          "falsePositiveRate": 0.007259528,
          "truePositiveCount": "2452",
          "falsePositiveCount": "4",
          "falseNegativeCount": "1636",
          "trueNegativeCount": "547",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9804444,
          "recall": 0.5900196,
          "precision": 0.99834436,
          "f1Score": 0.74169743,
          "falsePositiveRate": 0.007259528,
          "truePositiveCount": "2412",
          "falsePositiveCount": "4",
          "falseNegativeCount": "1676",
          "trueNegativeCount": "547",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9813577,
          "recall": 0.5799902,
          "precision": 0.9991572,
          "f1Score": 0.7339421,
          "falsePositiveRate": 0.003629764,
          "truePositiveCount": "2371",
          "falsePositiveCount": "2",
          "falseNegativeCount": "1717",
          "trueNegativeCount": "549",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9822546,
          "recall": 0.56996083,
          "precision": 0.99914235,
          "f1Score": 0.7258567,
          "falsePositiveRate": 0.003629764,
          "truePositiveCount": "2330",
          "falsePositiveCount": "2",
          "falseNegativeCount": "1758",
          "trueNegativeCount": "549",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9832565,
          "recall": 0.5599315,
          "precision": 0.99912703,
          "f1Score": 0.71766734,
          "falsePositiveRate": 0.003629764,
          "truePositiveCount": "2289",
          "falsePositiveCount": "2",
          "falseNegativeCount": "1799",
          "trueNegativeCount": "549",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.983971,
          "recall": 0.54990214,
          "precision": 0.99955535,
          "f1Score": 0.709484,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2248",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1840",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98478603,
          "recall": 0.54011744,
          "precision": 0.9995473,
          "f1Score": 0.7012863,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2208",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1880",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98576885,
          "recall": 0.52984345,
          "precision": 0.99953854,
          "f1Score": 0.6925659,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2166",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1922",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9865271,
          "recall": 0.5198141,
          "precision": 0.99952966,
          "f1Score": 0.6839395,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2125",
          "falsePositiveCount": "1",
          "falseNegativeCount": "1963",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98722655,
          "recall": 0.5100294,
          "precision": 0.9995206,
          "f1Score": 0.675413,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2085",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2003",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98776835,
          "recall": 0.49975538,
          "precision": 0.99951077,
          "f1Score": 0.66634053,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2043",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2045",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98843795,
          "recall": 0.49021527,
          "precision": 0.9995012,
          "f1Score": 0.657804,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "2004",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2084",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98898214,
          "recall": 0.4801859,
          "precision": 0.99949086,
          "f1Score": 0.64871114,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1963",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2125",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.98952276,
          "recall": 0.46991193,
          "precision": 0.9994797,
          "f1Score": 0.63926786,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1921",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2167",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99007314,
          "recall": 0.4598826,
          "precision": 0.9994684,
          "f1Score": 0.62992126,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1880",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2208",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.990457,
          "recall": 0.45009786,
          "precision": 0.9994568,
          "f1Score": 0.620678,
          "falsePositiveRate": 0.001814882,
          "truePositiveCount": "1840",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2248",
          "trueNegativeCount": "550",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99079466,
          "recall": 0.44006848,
          "precision": 1.0,
          "f1Score": 0.61117715,
          "truePositiveCount": "1799",
          "falseNegativeCount": "2289",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9911339,
          "recall": 0.43003914,
          "precision": 1.0,
          "f1Score": 0.60143685,
          "truePositiveCount": "1758",
          "falseNegativeCount": "2330",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99141955,
          "recall": 0.4200098,
          "precision": 1.0,
          "f1Score": 0.591559,
          "truePositiveCount": "1717",
          "falseNegativeCount": "2371",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9917892,
          "recall": 0.40998042,
          "precision": 1.0,
          "f1Score": 0.5815406,
          "truePositiveCount": "1676",
          "falseNegativeCount": "2412",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9921363,
          "recall": 0.39995107,
          "precision": 1.0,
          "f1Score": 0.57137865,
          "truePositiveCount": "1635",
          "falseNegativeCount": "2453",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99253094,
          "recall": 0.38992172,
          "precision": 1.0,
          "f1Score": 0.56107,
          "truePositiveCount": "1594",
          "falseNegativeCount": "2494",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9928577,
          "recall": 0.380137,
          "precision": 1.0,
          "f1Score": 0.5508685,
          "truePositiveCount": "1554",
          "falseNegativeCount": "2534",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99313205,
          "recall": 0.369863,
          "precision": 1.0,
          "f1Score": 0.54,
          "truePositiveCount": "1512",
          "falseNegativeCount": "2576",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9933454,
          "recall": 0.35983366,
          "precision": 1.0,
          "f1Score": 0.5292319,
          "truePositiveCount": "1471",
          "falseNegativeCount": "2617",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99355274,
          "recall": 0.35004893,
          "precision": 1.0,
          "f1Score": 0.5185722,
          "truePositiveCount": "1431",
          "falseNegativeCount": "2657",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99378794,
          "recall": 0.33977494,
          "precision": 1.0,
          "f1Score": 0.507212,
          "truePositiveCount": "1389",
          "falseNegativeCount": "2699",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9939992,
          "recall": 0.33023483,
          "precision": 1.0,
          "f1Score": 0.49650607,
          "truePositiveCount": "1350",
          "falseNegativeCount": "2738",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9942213,
          "recall": 0.31996086,
          "precision": 1.0,
          "f1Score": 0.48480356,
          "truePositiveCount": "1308",
          "falseNegativeCount": "2780",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9944552,
          "recall": 0.30993152,
          "precision": 1.0,
          "f1Score": 0.47320262,
          "truePositiveCount": "1267",
          "falseNegativeCount": "2821",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9946339,
          "recall": 0.29990214,
          "precision": 1.0,
          "f1Score": 0.46142265,
          "truePositiveCount": "1226",
          "falseNegativeCount": "2862",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99486357,
          "recall": 0.2898728,
          "precision": 1.0,
          "f1Score": 0.44945952,
          "truePositiveCount": "1185",
          "falseNegativeCount": "2903",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9950511,
          "recall": 0.28008807,
          "precision": 1.0,
          "f1Score": 0.4376075,
          "truePositiveCount": "1145",
          "falseNegativeCount": "2943",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9953063,
          "recall": 0.2698141,
          "precision": 1.0,
          "f1Score": 0.42496628,
          "truePositiveCount": "1103",
          "falseNegativeCount": "2985",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99548787,
          "recall": 0.26002935,
          "precision": 1.0,
          "f1Score": 0.4127354,
          "truePositiveCount": "1063",
          "falseNegativeCount": "3025",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99564606,
          "recall": 0.25024462,
          "precision": 1.0,
          "f1Score": 0.40031305,
          "truePositiveCount": "1023",
          "falseNegativeCount": "3065",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9958494,
          "recall": 0.23997064,
          "precision": 1.0,
          "f1Score": 0.3870586,
          "truePositiveCount": "981",
          "falseNegativeCount": "3107",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9959897,
          "recall": 0.2299413,
          "precision": 1.0,
          "f1Score": 0.37390614,
          "truePositiveCount": "940",
          "falseNegativeCount": "3148",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9961461,
          "recall": 0.21991193,
          "precision": 1.0,
          "f1Score": 0.3605374,
          "truePositiveCount": "899",
          "falseNegativeCount": "3189",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99626255,
          "recall": 0.20988259,
          "precision": 1.0,
          "f1Score": 0.346947,
          "truePositiveCount": "858",
          "falseNegativeCount": "3230",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99635655,
          "recall": 0.19985323,
          "precision": 1.0,
          "f1Score": 0.33312947,
          "truePositiveCount": "817",
          "falseNegativeCount": "3271",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9964815,
          "recall": 0.1900685,
          "precision": 1.0,
          "f1Score": 0.31942445,
          "truePositiveCount": "777",
          "falseNegativeCount": "3311",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9966108,
          "recall": 0.17979452,
          "precision": 1.0,
          "f1Score": 0.30478954,
          "truePositiveCount": "735",
          "falseNegativeCount": "3353",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99673027,
          "recall": 0.16976516,
          "precision": 1.0,
          "f1Score": 0.29025513,
          "truePositiveCount": "694",
          "falseNegativeCount": "3394",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99681914,
          "recall": 0.15998043,
          "precision": 1.0,
          "f1Score": 0.27583298,
          "truePositiveCount": "654",
          "falseNegativeCount": "3434",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9969455,
          "recall": 0.15019569,
          "precision": 1.0,
          "f1Score": 0.26116547,
          "truePositiveCount": "614",
          "falseNegativeCount": "3474",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99705213,
          "recall": 0.13992172,
          "precision": 1.0,
          "f1Score": 0.24549356,
          "truePositiveCount": "572",
          "falseNegativeCount": "3516",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9971576,
          "recall": 0.13013698,
          "precision": 1.0,
          "f1Score": 0.23030303,
          "truePositiveCount": "532",
          "falseNegativeCount": "3556",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9972461,
          "recall": 0.11986301,
          "precision": 1.0,
          "f1Score": 0.21406728,
          "truePositiveCount": "490",
          "falseNegativeCount": "3598",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9973425,
          "recall": 0.110078275,
          "precision": 1.0,
          "f1Score": 0.19832525,
          "truePositiveCount": "450",
          "falseNegativeCount": "3638",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9974355,
          "recall": 0.10004892,
          "precision": 1.0,
          "f1Score": 0.18189904,
          "truePositiveCount": "409",
          "falseNegativeCount": "3679",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9975045,
          "recall": 0.09001957,
          "precision": 1.0,
          "f1Score": 0.16517055,
          "truePositiveCount": "368",
          "falseNegativeCount": "3720",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99760437,
          "recall": 0.079990216,
          "precision": 1.0,
          "f1Score": 0.14813137,
          "truePositiveCount": "327",
          "falseNegativeCount": "3761",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9976864,
          "recall": 0.07020548,
          "precision": 1.0,
          "f1Score": 0.1312,
          "truePositiveCount": "287",
          "falseNegativeCount": "3801",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9977548,
          "recall": 0.059931505,
          "precision": 1.0,
          "f1Score": 0.11308562,
          "truePositiveCount": "245",
          "falseNegativeCount": "3843",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.99784184,
          "recall": 0.05014677,
          "precision": 1.0,
          "f1Score": 0.095504306,
          "truePositiveCount": "205",
          "falseNegativeCount": "3883",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9979226,
          "recall": 0.040117417,
          "precision": 1.0,
          "f1Score": 0.07714017,
          "truePositiveCount": "164",
          "falseNegativeCount": "3924",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9980229,
          "recall": 0.030088063,
          "precision": 1.0,
          "f1Score": 0.058418427,
          "truePositiveCount": "123",
          "falseNegativeCount": "3965",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9981275,
          "recall": 0.01981409,
          "precision": 1.0,
          "f1Score": 0.03885824,
          "truePositiveCount": "81",
          "falseNegativeCount": "4007",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9982374,
          "recall": 0.009784736,
          "precision": 1.0,
          "f1Score": 0.019379845,
          "truePositiveCount": "40",
          "falseNegativeCount": "4048",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9987167,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4088",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 1.0,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4088",
          "trueNegativeCount": "551",
          "positionThreshold": 1
        }
      ],
      "auRoc": 0.9325568,
      "logLoss": 0.09388557
    },
    "displayName": "1"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/6226491855750944430",
    "annotationSpecId": "not available",
    "createTime": "2021-02-28T17:24:31.800758Z",
    "evaluatedExampleCount": 551,
    "classificationEvaluationMetrics": {
      "auPrc": 0.34112322,
      "confidenceMetricsEntry": [
        {
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.009267787,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.022539802,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.03954769,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.045538254,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.050852664,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.063637815,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.07210142,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.07750216,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.09185968,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.09674809,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.10816234,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.12236822,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.1267207,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.15273023,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.1649416,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.18867286,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.19824302,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.20855853,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.2164171,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.2228882,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.2343938,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.24437484,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.24899487,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.25527748,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.2643137,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.27015588,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.2785622,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.29147997,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.30125153,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.30796623,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.32498023,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.33223236,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.3348615,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.33746937,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.34359238,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.3495217,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.35605764,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.36666742,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.37787727,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.39064476,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.39657614,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.39991632,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.41141164,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.4184669,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.42953506,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.43759817,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.44597256,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.44895133,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.46139872,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.47061405,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.48159525,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.48736593,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.4939781,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.4995014,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.46823958,
          "precision": 0.63703704,
          "f1Score": 0.53974897,
          "falsePositiveRate": 0.035958905,
          "truePositiveCount": "258",
          "falsePositiveCount": "147",
          "falseNegativeCount": "293",
          "trueNegativeCount": "3941",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5025096,
          "recall": 0.45916516,
          "precision": 0.63408524,
          "f1Score": 0.5326316,
          "falsePositiveRate": 0.035714287,
          "truePositiveCount": "253",
          "falsePositiveCount": "146",
          "falseNegativeCount": "298",
          "trueNegativeCount": "3942",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.51137525,
          "recall": 0.44827586,
          "precision": 0.63010204,
          "f1Score": 0.52386004,
          "falsePositiveRate": 0.035469666,
          "truePositiveCount": "247",
          "falsePositiveCount": "145",
          "falseNegativeCount": "304",
          "trueNegativeCount": "3943",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.51734245,
          "recall": 0.43920144,
          "precision": 0.62857145,
          "f1Score": 0.517094,
          "falsePositiveRate": 0.03498043,
          "truePositiveCount": "242",
          "falsePositiveCount": "143",
          "falseNegativeCount": "309",
          "trueNegativeCount": "3945",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5248498,
          "recall": 0.42831215,
          "precision": 0.631016,
          "f1Score": 0.5102703,
          "falsePositiveRate": 0.03375734,
          "truePositiveCount": "236",
          "falsePositiveCount": "138",
          "falseNegativeCount": "315",
          "trueNegativeCount": "3950",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.53241414,
          "recall": 0.41923776,
          "precision": 0.64705884,
          "f1Score": 0.5088106,
          "falsePositiveRate": 0.030821918,
          "truePositiveCount": "231",
          "falsePositiveCount": "126",
          "falseNegativeCount": "320",
          "trueNegativeCount": "3962",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5427009,
          "recall": 0.40834847,
          "precision": 0.64841497,
          "f1Score": 0.5011136,
          "falsePositiveRate": 0.029843444,
          "truePositiveCount": "225",
          "falsePositiveCount": "122",
          "falseNegativeCount": "326",
          "trueNegativeCount": "3966",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5505563,
          "recall": 0.39927405,
          "precision": 0.6547619,
          "f1Score": 0.4960541,
          "falsePositiveRate": 0.028375734,
          "truePositiveCount": "220",
          "falsePositiveCount": "116",
          "falseNegativeCount": "331",
          "trueNegativeCount": "3972",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.55334836,
          "recall": 0.38838476,
          "precision": 0.65045595,
          "f1Score": 0.48636365,
          "falsePositiveRate": 0.028131116,
          "truePositiveCount": "214",
          "falsePositiveCount": "115",
          "falseNegativeCount": "337",
          "trueNegativeCount": "3973",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.55837405,
          "recall": 0.37931034,
          "precision": 0.6490683,
          "f1Score": 0.4788087,
          "falsePositiveRate": 0.02764188,
          "truePositiveCount": "209",
          "falsePositiveCount": "113",
          "falseNegativeCount": "342",
          "trueNegativeCount": "3975",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5685674,
          "recall": 0.36842105,
          "precision": 0.650641,
          "f1Score": 0.47045192,
          "falsePositiveRate": 0.026663406,
          "truePositiveCount": "203",
          "falsePositiveCount": "109",
          "falseNegativeCount": "348",
          "trueNegativeCount": "3979",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5732897,
          "recall": 0.35934663,
          "precision": 0.6578073,
          "f1Score": 0.46478873,
          "falsePositiveRate": 0.025195695,
          "truePositiveCount": "198",
          "falsePositiveCount": "103",
          "falseNegativeCount": "353",
          "trueNegativeCount": "3985",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5798386,
          "recall": 0.34845734,
          "precision": 0.65753424,
          "f1Score": 0.455516,
          "falsePositiveRate": 0.02446184,
          "truePositiveCount": "192",
          "falsePositiveCount": "100",
          "falseNegativeCount": "359",
          "trueNegativeCount": "3988",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.5841479,
          "recall": 0.33938295,
          "precision": 0.6561403,
          "f1Score": 0.4473684,
          "falsePositiveRate": 0.023972603,
          "truePositiveCount": "187",
          "falsePositiveCount": "98",
          "falseNegativeCount": "364",
          "trueNegativeCount": "3990",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.58893955,
          "recall": 0.32849365,
          "precision": 0.6630037,
          "f1Score": 0.4393204,
          "falsePositiveRate": 0.022504892,
          "truePositiveCount": "181",
          "falsePositiveCount": "92",
          "falseNegativeCount": "370",
          "trueNegativeCount": "3996",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.59818196,
          "recall": 0.31941923,
          "precision": 0.6743295,
          "f1Score": 0.43349755,
          "falsePositiveRate": 0.020792564,
          "truePositiveCount": "176",
          "falsePositiveCount": "85",
          "falseNegativeCount": "375",
          "trueNegativeCount": "4003",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.60704446,
          "recall": 0.30852994,
          "precision": 0.6854839,
          "f1Score": 0.42553192,
          "falsePositiveRate": 0.019080235,
          "truePositiveCount": "170",
          "falsePositiveCount": "78",
          "falseNegativeCount": "381",
          "trueNegativeCount": "4010",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6096991,
          "recall": 0.29945552,
          "precision": 0.6818182,
          "f1Score": 0.41614124,
          "falsePositiveRate": 0.018835617,
          "truePositiveCount": "165",
          "falsePositiveCount": "77",
          "falseNegativeCount": "386",
          "trueNegativeCount": "4011",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6194289,
          "recall": 0.28856623,
          "precision": 0.6913043,
          "f1Score": 0.4071703,
          "falsePositiveRate": 0.017367907,
          "truePositiveCount": "159",
          "falsePositiveCount": "71",
          "falseNegativeCount": "392",
          "trueNegativeCount": "4017",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6299003,
          "recall": 0.27949184,
          "precision": 0.6936937,
          "f1Score": 0.3984476,
          "falsePositiveRate": 0.01663405,
          "truePositiveCount": "154",
          "falsePositiveCount": "68",
          "falseNegativeCount": "397",
          "trueNegativeCount": "4020",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6326805,
          "recall": 0.26860255,
          "precision": 0.6883721,
          "f1Score": 0.38642296,
          "falsePositiveRate": 0.016389433,
          "truePositiveCount": "148",
          "falsePositiveCount": "67",
          "falseNegativeCount": "403",
          "trueNegativeCount": "4021",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6446329,
          "recall": 0.25952813,
          "precision": 0.71859294,
          "f1Score": 0.38133332,
          "falsePositiveRate": 0.01369863,
          "truePositiveCount": "143",
          "falsePositiveCount": "56",
          "falseNegativeCount": "408",
          "trueNegativeCount": "4032",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6525863,
          "recall": 0.24863884,
          "precision": 0.7287234,
          "f1Score": 0.37077132,
          "falsePositiveRate": 0.012475538,
          "truePositiveCount": "137",
          "falsePositiveCount": "51",
          "falseNegativeCount": "414",
          "trueNegativeCount": "4037",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6613302,
          "recall": 0.23956443,
          "precision": 0.73333335,
          "f1Score": 0.3611491,
          "falsePositiveRate": 0.011741683,
          "truePositiveCount": "132",
          "falsePositiveCount": "48",
          "falseNegativeCount": "419",
          "trueNegativeCount": "4040",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6681088,
          "recall": 0.22867514,
          "precision": 0.73255813,
          "f1Score": 0.34854773,
          "falsePositiveRate": 0.011252446,
          "truePositiveCount": "126",
          "falsePositiveCount": "46",
          "falseNegativeCount": "425",
          "trueNegativeCount": "4042",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.6787454,
          "recall": 0.21960072,
          "precision": 0.7378049,
          "f1Score": 0.33846155,
          "falsePositiveRate": 0.010518591,
          "truePositiveCount": "121",
          "falsePositiveCount": "43",
          "falseNegativeCount": "430",
          "trueNegativeCount": "4045",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.68742466,
          "recall": 0.20871143,
          "precision": 0.73717946,
          "f1Score": 0.32531825,
          "falsePositiveRate": 0.010029354,
          "truePositiveCount": "115",
          "falsePositiveCount": "41",
          "falseNegativeCount": "436",
          "trueNegativeCount": "4047",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.69688004,
          "recall": 0.19963703,
          "precision": 0.7482993,
          "f1Score": 0.31518623,
          "falsePositiveRate": 0.009050881,
          "truePositiveCount": "110",
          "falsePositiveCount": "37",
          "falseNegativeCount": "441",
          "trueNegativeCount": "4051",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.70484525,
          "recall": 0.18874773,
          "precision": 0.7647059,
          "f1Score": 0.30276564,
          "falsePositiveRate": 0.007827789,
          "truePositiveCount": "104",
          "falsePositiveCount": "32",
          "falseNegativeCount": "447",
          "trueNegativeCount": "4056",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.71349317,
          "recall": 0.17967331,
          "precision": 0.76153845,
          "f1Score": 0.2907489,
          "falsePositiveRate": 0.00758317,
          "truePositiveCount": "99",
          "falsePositiveCount": "31",
          "falseNegativeCount": "452",
          "trueNegativeCount": "4057",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.72145003,
          "recall": 0.16878402,
          "precision": 0.75609756,
          "f1Score": 0.27596438,
          "falsePositiveRate": 0.007338552,
          "truePositiveCount": "93",
          "falsePositiveCount": "30",
          "falseNegativeCount": "458",
          "trueNegativeCount": "4058",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.72871804,
          "recall": 0.15970962,
          "precision": 0.76521736,
          "f1Score": 0.26426426,
          "falsePositiveRate": 0.0066046966,
          "truePositiveCount": "88",
          "falsePositiveCount": "27",
          "falseNegativeCount": "463",
          "trueNegativeCount": "4061",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.74161375,
          "recall": 0.14882033,
          "precision": 0.76635516,
          "f1Score": 0.24924012,
          "falsePositiveRate": 0.00611546,
          "truePositiveCount": "82",
          "falsePositiveCount": "25",
          "falseNegativeCount": "469",
          "trueNegativeCount": "4063",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.74766874,
          "recall": 0.13974592,
          "precision": 0.78571427,
          "f1Score": 0.23728813,
          "falsePositiveRate": 0.0051369863,
          "truePositiveCount": "77",
          "falsePositiveCount": "21",
          "falseNegativeCount": "474",
          "trueNegativeCount": "4067",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.75845456,
          "recall": 0.12885663,
          "precision": 0.8068182,
          "f1Score": 0.22222222,
          "falsePositiveRate": 0.0041585127,
          "truePositiveCount": "71",
          "falsePositiveCount": "17",
          "falseNegativeCount": "480",
          "trueNegativeCount": "4071",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7666716,
          "recall": 0.11978222,
          "precision": 0.80487806,
          "f1Score": 0.2085308,
          "falsePositiveRate": 0.0039138943,
          "truePositiveCount": "66",
          "falsePositiveCount": "16",
          "falseNegativeCount": "485",
          "trueNegativeCount": "4072",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.7772814,
          "recall": 0.108892925,
          "precision": 0.7894737,
          "f1Score": 0.19138756,
          "falsePositiveRate": 0.0039138943,
          "truePositiveCount": "60",
          "falsePositiveCount": "16",
          "falseNegativeCount": "491",
          "trueNegativeCount": "4072",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.78661436,
          "recall": 0.09981851,
          "precision": 0.8088235,
          "f1Score": 0.17770597,
          "falsePositiveRate": 0.003180039,
          "truePositiveCount": "55",
          "falsePositiveCount": "13",
          "falseNegativeCount": "496",
          "trueNegativeCount": "4075",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.8050728,
          "recall": 0.08892922,
          "precision": 0.8448276,
          "f1Score": 0.16091955,
          "falsePositiveRate": 0.0022015655,
          "truePositiveCount": "49",
          "falsePositiveCount": "9",
          "falseNegativeCount": "502",
          "trueNegativeCount": "4079",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.8127946,
          "recall": 0.07985481,
          "precision": 0.8301887,
          "f1Score": 0.14569536,
          "falsePositiveRate": 0.0022015655,
          "truePositiveCount": "44",
          "falsePositiveCount": "9",
          "falseNegativeCount": "507",
          "trueNegativeCount": "4079",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.82794523,
          "recall": 0.06896552,
          "precision": 0.82608694,
          "f1Score": 0.12730318,
          "falsePositiveRate": 0.0019569471,
          "truePositiveCount": "38",
          "falsePositiveCount": "8",
          "falseNegativeCount": "513",
          "trueNegativeCount": "4080",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.83858025,
          "recall": 0.05989111,
          "precision": 0.84615386,
          "f1Score": 0.11186441,
          "falsePositiveRate": 0.0014677104,
          "truePositiveCount": "33",
          "falsePositiveCount": "6",
          "falseNegativeCount": "518",
          "trueNegativeCount": "4082",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.85226774,
          "recall": 0.049001817,
          "precision": 0.87096775,
          "f1Score": 0.0927835,
          "falsePositiveRate": 0.0009784736,
          "truePositiveCount": "27",
          "falsePositiveCount": "4",
          "falseNegativeCount": "524",
          "trueNegativeCount": "4084",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.86955357,
          "recall": 0.039927404,
          "precision": 0.88,
          "f1Score": 0.07638889,
          "falsePositiveRate": 0.0007338552,
          "truePositiveCount": "22",
          "falsePositiveCount": "3",
          "falseNegativeCount": "529",
          "trueNegativeCount": "4085",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.8754055,
          "recall": 0.029038113,
          "precision": 0.84210527,
          "f1Score": 0.056140352,
          "falsePositiveRate": 0.0007338552,
          "truePositiveCount": "16",
          "falsePositiveCount": "3",
          "falseNegativeCount": "535",
          "trueNegativeCount": "4085",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.888999,
          "recall": 0.019963702,
          "precision": 0.78571427,
          "f1Score": 0.038938053,
          "falsePositiveRate": 0.0007338552,
          "truePositiveCount": "11",
          "falsePositiveCount": "3",
          "falseNegativeCount": "540",
          "trueNegativeCount": "4085",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.912919,
          "recall": 0.00907441,
          "precision": 0.71428573,
          "f1Score": 0.017921148,
          "falsePositiveRate": 0.0004892368,
          "truePositiveCount": "5",
          "falsePositiveCount": "2",
          "falseNegativeCount": "546",
          "trueNegativeCount": "4086",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 0.9486238,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "551",
          "trueNegativeCount": "4088",
          "positionThreshold": 1
        },
        {
          "confidenceThreshold": 1.0,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "551",
          "trueNegativeCount": "4088",
          "positionThreshold": 1
        }
      ],
      "auRoc": 0.71778876,
      "logLoss": 0.993795
    },
    "displayName": "2"
  },
  {
    "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/7171857336139058652",
    "annotationSpecId": "not available",
    "createTime": "2021-02-28T17:24:31.800758Z",
    "evaluatedExampleCount": 4639,
    "classificationEvaluationMetrics": {
      "auPrc": 0.97598535,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.5,
          "f1Score": 0.6666667,
          "falsePositiveRate": 1.0,
          "truePositiveCount": "4639",
          "falsePositiveCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.009267787,
          "recall": 0.9997844,
          "precision": 0.6205512,
          "f1Score": 0.7657888,
          "falsePositiveRate": 0.6113387,
          "truePositiveCount": "4638",
          "falsePositiveCount": "2836",
          "falseNegativeCount": "1",
          "trueNegativeCount": "1803",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.08771089,
          "recall": 0.99029964,
          "precision": 0.77640694,
          "f1Score": 0.87040544,
          "falsePositiveRate": 0.28519076,
          "truePositiveCount": "4594",
          "falsePositiveCount": "1323",
          "falseNegativeCount": "45",
          "trueNegativeCount": "3316",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.18867286,
          "recall": 0.98016816,
          "precision": 0.82268864,
          "f1Score": 0.89455044,
          "falsePositiveRate": 0.21125242,
          "truePositiveCount": "4547",
          "falsePositiveCount": "980",
          "falseNegativeCount": "92",
          "trueNegativeCount": "3659",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.24711765,
          "recall": 0.9702522,
          "precision": 0.841623,
          "f1Score": 0.9013718,
          "falsePositiveRate": 0.18258245,
          "truePositiveCount": "4501",
          "falsePositiveCount": "847",
          "falseNegativeCount": "138",
          "trueNegativeCount": "3792",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.29667678,
          "recall": 0.96012074,
          "precision": 0.85358375,
          "f1Score": 0.90372324,
          "falsePositiveRate": 0.16469067,
          "truePositiveCount": "4454",
          "falsePositiveCount": "764",
          "falseNegativeCount": "185",
          "trueNegativeCount": "3875",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.33910435,
          "recall": 0.9499892,
          "precision": 0.8619206,
          "f1Score": 0.9038146,
          "falsePositiveRate": 0.15218797,
          "truePositiveCount": "4407",
          "falsePositiveCount": "706",
          "falseNegativeCount": "232",
          "trueNegativeCount": "3933",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.3722213,
          "recall": 0.94028884,
          "precision": 0.87031126,
          "f1Score": 0.9039478,
          "falsePositiveRate": 0.14011641,
          "truePositiveCount": "4362",
          "falsePositiveCount": "650",
          "falseNegativeCount": "277",
          "trueNegativeCount": "3989",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.41141066,
          "recall": 0.93015736,
          "precision": 0.87971455,
          "f1Score": 0.90423304,
          "falsePositiveRate": 0.12718259,
          "truePositiveCount": "4315",
          "falsePositiveCount": "590",
          "falseNegativeCount": "324",
          "trueNegativeCount": "4049",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.44597256,
          "recall": 0.920457,
          "precision": 0.88921285,
          "f1Score": 0.9045652,
          "falsePositiveRate": 0.11467989,
          "truePositiveCount": "4270",
          "falsePositiveCount": "532",
          "falseNegativeCount": "369",
          "trueNegativeCount": "4107",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.48088387,
          "recall": 0.91010994,
          "precision": 0.89829785,
          "f1Score": 0.9041653,
          "falsePositiveRate": 0.10303945,
          "truePositiveCount": "4222",
          "falsePositiveCount": "478",
          "falseNegativeCount": "417",
          "trueNegativeCount": "4161",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5,
          "recall": 0.90515196,
          "precision": 0.90515196,
          "f1Score": 0.90515196,
          "falsePositiveRate": 0.09484803,
          "truePositiveCount": "4199",
          "falsePositiveCount": "440",
          "falseNegativeCount": "440",
          "trueNegativeCount": "4199",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.51082355,
          "recall": 0.9006251,
          "precision": 0.90688086,
          "f1Score": 0.90374213,
          "falsePositiveRate": 0.09247683,
          "truePositiveCount": "4178",
          "falsePositiveCount": "429",
          "falseNegativeCount": "461",
          "trueNegativeCount": "4210",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.5395309,
          "recall": 0.8902781,
          "precision": 0.9145261,
          "f1Score": 0.9022392,
          "falsePositiveRate": 0.083207585,
          "truePositiveCount": "4130",
          "falsePositiveCount": "386",
          "falseNegativeCount": "509",
          "trueNegativeCount": "4253",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.569127,
          "recall": 0.88014656,
          "precision": 0.92083895,
          "f1Score": 0.90003306,
          "falsePositiveRate": 0.07566286,
          "truePositiveCount": "4083",
          "falsePositiveCount": "351",
          "falseNegativeCount": "556",
          "trueNegativeCount": "4288",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.59819055,
          "recall": 0.8700151,
          "precision": 0.9284564,
          "f1Score": 0.8982862,
          "falsePositiveRate": 0.06704031,
          "truePositiveCount": "4036",
          "falsePositiveCount": "311",
          "falseNegativeCount": "603",
          "trueNegativeCount": "4328",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6267493,
          "recall": 0.86009914,
          "precision": 0.9350832,
          "f1Score": 0.8960252,
          "falsePositiveRate": 0.059711143,
          "truePositiveCount": "3990",
          "falsePositiveCount": "277",
          "falseNegativeCount": "649",
          "trueNegativeCount": "4362",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.6483357,
          "recall": 0.84996766,
          "precision": 0.9401526,
          "f1Score": 0.8927884,
          "falsePositiveRate": 0.05410649,
          "truePositiveCount": "3943",
          "falsePositiveCount": "251",
          "falseNegativeCount": "696",
          "trueNegativeCount": "4388",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.68742466,
          "recall": 0.8404829,
          "precision": 0.95051193,
          "f1Score": 0.8921176,
          "falsePositiveRate": 0.04375943,
          "truePositiveCount": "3899",
          "falsePositiveCount": "203",
          "falseNegativeCount": "740",
          "trueNegativeCount": "4436",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7178758,
          "recall": 0.83035135,
          "precision": 0.95630586,
          "f1Score": 0.8888889,
          "falsePositiveRate": 0.03793921,
          "truePositiveCount": "3852",
          "falsePositiveCount": "176",
          "falseNegativeCount": "787",
          "trueNegativeCount": "4463",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7454512,
          "recall": 0.8200043,
          "precision": 0.9620637,
          "f1Score": 0.8853718,
          "falsePositiveRate": 0.032334555,
          "truePositiveCount": "3804",
          "falsePositiveCount": "150",
          "falseNegativeCount": "835",
          "trueNegativeCount": "4489",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.7702336,
          "recall": 0.8105195,
          "precision": 0.9685729,
          "f1Score": 0.8825255,
          "falsePositiveRate": 0.02629877,
          "truePositiveCount": "3760",
          "falsePositiveCount": "122",
          "falseNegativeCount": "879",
          "trueNegativeCount": "4517",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.79181457,
          "recall": 0.80038804,
          "precision": 0.972244,
          "f1Score": 0.87798536,
          "falsePositiveRate": 0.022849752,
          "truePositiveCount": "3713",
          "falsePositiveCount": "106",
          "falseNegativeCount": "926",
          "trueNegativeCount": "4533",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.80905926,
          "recall": 0.7902565,
          "precision": 0.9755189,
          "f1Score": 0.873169,
          "falsePositiveRate": 0.01983186,
          "truePositiveCount": "3666",
          "falsePositiveCount": "92",
          "falseNegativeCount": "973",
          "trueNegativeCount": "4547",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.82475233,
          "recall": 0.7803406,
          "precision": 0.97574127,
          "f1Score": 0.86716974,
          "falsePositiveRate": 0.019400733,
          "truePositiveCount": "3620",
          "falsePositiveCount": "90",
          "falseNegativeCount": "1019",
          "trueNegativeCount": "4549",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.84347624,
          "recall": 0.7702091,
          "precision": 0.97863597,
          "f1Score": 0.86200243,
          "falsePositiveRate": 0.01681397,
          "truePositiveCount": "3573",
          "falsePositiveCount": "78",
          "falseNegativeCount": "1066",
          "trueNegativeCount": "4561",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.8612723,
          "recall": 0.7602932,
          "precision": 0.9799944,
          "f1Score": 0.8562758,
          "falsePositiveRate": 0.015520587,
          "truePositiveCount": "3527",
          "falsePositiveCount": "72",
          "falseNegativeCount": "1112",
          "trueNegativeCount": "4567",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.87450475,
          "recall": 0.75016165,
          "precision": 0.9811108,
          "f1Score": 0.8502321,
          "falsePositiveRate": 0.014442768,
          "truePositiveCount": "3480",
          "falsePositiveCount": "67",
          "falseNegativeCount": "1159",
          "trueNegativeCount": "4572",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.8876167,
          "recall": 0.74003017,
          "precision": 0.9825415,
          "f1Score": 0.8442149,
          "falsePositiveRate": 0.013149385,
          "truePositiveCount": "3433",
          "falsePositiveCount": "61",
          "falseNegativeCount": "1206",
          "trueNegativeCount": "4578",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.89729506,
          "recall": 0.7303298,
          "precision": 0.98402554,
          "f1Score": 0.8384063,
          "falsePositiveRate": 0.011856004,
          "truePositiveCount": "3388",
          "falsePositiveCount": "55",
          "falseNegativeCount": "1251",
          "trueNegativeCount": "4584",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9069939,
          "recall": 0.72041386,
          "precision": 0.9858407,
          "f1Score": 0.8324823,
          "falsePositiveRate": 0.010347057,
          "truePositiveCount": "3342",
          "falsePositiveCount": "48",
          "falseNegativeCount": "1297",
          "trueNegativeCount": "4591",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.91544837,
          "recall": 0.7102824,
          "precision": 0.987118,
          "f1Score": 0.8261251,
          "falsePositiveRate": 0.009269239,
          "truePositiveCount": "3295",
          "falsePositiveCount": "43",
          "falseNegativeCount": "1344",
          "trueNegativeCount": "4596",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9246296,
          "recall": 0.700582,
          "precision": 0.9896468,
          "f1Score": 0.8203963,
          "falsePositiveRate": 0.007329166,
          "truePositiveCount": "3250",
          "falsePositiveCount": "34",
          "falseNegativeCount": "1389",
          "trueNegativeCount": "4605",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.932719,
          "recall": 0.6900194,
          "precision": 0.99071497,
          "f1Score": 0.8134689,
          "falsePositiveRate": 0.006466911,
          "truePositiveCount": "3201",
          "falsePositiveCount": "30",
          "falseNegativeCount": "1438",
          "trueNegativeCount": "4609",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.93940556,
          "recall": 0.6807502,
          "precision": 0.9921458,
          "f1Score": 0.80746615,
          "falsePositiveRate": 0.0053890925,
          "truePositiveCount": "3158",
          "falsePositiveCount": "25",
          "falseNegativeCount": "1481",
          "trueNegativeCount": "4614",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9436984,
          "recall": 0.6704031,
          "precision": 0.99234205,
          "f1Score": 0.8002058,
          "falsePositiveRate": 0.0051735286,
          "truePositiveCount": "3110",
          "falsePositiveCount": "24",
          "falseNegativeCount": "1529",
          "trueNegativeCount": "4615",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9486017,
          "recall": 0.6602716,
          "precision": 0.992547,
          "f1Score": 0.7930097,
          "falsePositiveRate": 0.004957965,
          "truePositiveCount": "3063",
          "falsePositiveCount": "23",
          "falseNegativeCount": "1576",
          "trueNegativeCount": "4616",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.95302504,
          "recall": 0.6501401,
          "precision": 0.9937397,
          "f1Score": 0.78603077,
          "falsePositiveRate": 0.0040957103,
          "truePositiveCount": "3016",
          "falsePositiveCount": "19",
          "falseNegativeCount": "1623",
          "trueNegativeCount": "4620",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.95584136,
          "recall": 0.6400086,
          "precision": 0.99530673,
          "f1Score": 0.7790606,
          "falsePositiveRate": 0.003017892,
          "truePositiveCount": "2969",
          "falsePositiveCount": "14",
          "falseNegativeCount": "1670",
          "trueNegativeCount": "4625",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.95881057,
          "recall": 0.6300927,
          "precision": 0.9955722,
          "f1Score": 0.7717492,
          "falsePositiveRate": 0.002802328,
          "truePositiveCount": "2923",
          "falsePositiveCount": "13",
          "falseNegativeCount": "1716",
          "trueNegativeCount": "4626",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96166563,
          "recall": 0.62082344,
          "precision": 0.9961951,
          "f1Score": 0.76494026,
          "falsePositiveRate": 0.0023712006,
          "truePositiveCount": "2880",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1759",
          "trueNegativeCount": "4628",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96420497,
          "recall": 0.61026084,
          "precision": 0.9961295,
          "f1Score": 0.75685066,
          "falsePositiveRate": 0.0023712006,
          "truePositiveCount": "2831",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1808",
          "trueNegativeCount": "4628",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96650803,
          "recall": 0.5999138,
          "precision": 0.996063,
          "f1Score": 0.7488228,
          "falsePositiveRate": 0.0023712006,
          "truePositiveCount": "2783",
          "falsePositiveCount": "11",
          "falseNegativeCount": "1856",
          "trueNegativeCount": "4628",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.96897614,
          "recall": 0.59042895,
          "precision": 0.9963623,
          "f1Score": 0.74147266,
          "falsePositiveRate": 0.002155637,
          "truePositiveCount": "2739",
          "falsePositiveCount": "10",
          "falseNegativeCount": "1900",
          "trueNegativeCount": "4629",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.97092766,
          "recall": 0.5807286,
          "precision": 0.99667037,
          "f1Score": 0.73385996,
          "falsePositiveRate": 0.0019400733,
          "truePositiveCount": "2694",
          "falsePositiveCount": "9",
          "falseNegativeCount": "1945",
          "trueNegativeCount": "4630",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9727514,
          "recall": 0.5699504,
          "precision": 0.9969834,
          "f1Score": 0.7252777,
          "falsePositiveRate": 0.0017245096,
          "truePositiveCount": "2644",
          "falsePositiveCount": "8",
          "falseNegativeCount": "1995",
          "trueNegativeCount": "4631",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9747563,
          "recall": 0.56046563,
          "precision": 0.99731493,
          "f1Score": 0.7176373,
          "falsePositiveRate": 0.001508946,
          "truePositiveCount": "2600",
          "falsePositiveCount": "7",
          "falseNegativeCount": "2039",
          "trueNegativeCount": "4632",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9766838,
          "recall": 0.549903,
          "precision": 0.9976535,
          "f1Score": 0.709005,
          "falsePositiveRate": 0.0012933821,
          "truePositiveCount": "2551",
          "falsePositiveCount": "6",
          "falseNegativeCount": "2088",
          "trueNegativeCount": "4633",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.97776866,
          "recall": 0.54063374,
          "precision": 0.99801034,
          "f1Score": 0.7013423,
          "falsePositiveRate": 0.0010778185,
          "truePositiveCount": "2508",
          "falsePositiveCount": "5",
          "falseNegativeCount": "2131",
          "trueNegativeCount": "4634",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9792647,
          "recall": 0.53050226,
          "precision": 0.9979724,
          "f1Score": 0.6927516,
          "falsePositiveRate": 0.0010778185,
          "truePositiveCount": "2461",
          "falsePositiveCount": "5",
          "falseNegativeCount": "2178",
          "trueNegativeCount": "4634",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9804138,
          "recall": 0.5203708,
          "precision": 0.99834573,
          "f1Score": 0.6841434,
          "falsePositiveRate": 0.0008622548,
          "truePositiveCount": "2414",
          "falsePositiveCount": "4",
          "falseNegativeCount": "2225",
          "trueNegativeCount": "4635",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9814273,
          "recall": 0.51045483,
          "precision": 0.9991561,
          "f1Score": 0.6757027,
          "falsePositiveRate": 0.0004311274,
          "truePositiveCount": "2368",
          "falsePositiveCount": "2",
          "falseNegativeCount": "2271",
          "trueNegativeCount": "4637",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98243207,
          "recall": 0.5005389,
          "precision": 0.9991394,
          "f1Score": 0.6669539,
          "falsePositiveRate": 0.0004311274,
          "truePositiveCount": "2322",
          "falsePositiveCount": "2",
          "falseNegativeCount": "2317",
          "trueNegativeCount": "4637",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9835001,
          "recall": 0.49019185,
          "precision": 0.9995604,
          "f1Score": 0.6577958,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2274",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2365",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9844133,
          "recall": 0.4804915,
          "precision": 0.9995516,
          "f1Score": 0.6490028,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2229",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2410",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98528725,
          "recall": 0.47079113,
          "precision": 0.99954236,
          "f1Score": 0.6400938,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2184",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2455",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.98630506,
          "recall": 0.46001294,
          "precision": 0.9995316,
          "f1Score": 0.6300561,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2134",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2505",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9871925,
          "recall": 0.44988143,
          "precision": 0.9995211,
          "f1Score": 0.6204846,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2087",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2552",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9877509,
          "recall": 0.44082776,
          "precision": 0.99951124,
          "f1Score": 0.6118175,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "2045",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2594",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9885219,
          "recall": 0.43048072,
          "precision": 0.9994995,
          "f1Score": 0.6017779,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1997",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2642",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9892192,
          "recall": 0.4199181,
          "precision": 0.9994869,
          "f1Score": 0.5913783,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1948",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2691",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9897657,
          "recall": 0.41000214,
          "precision": 0.9994745,
          "f1Score": 0.5814735,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1902",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2737",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9902156,
          "recall": 0.40051734,
          "precision": 0.99946207,
          "f1Score": 0.57186824,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1858",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2781",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.990671,
          "recall": 0.3901703,
          "precision": 0.9994478,
          "f1Score": 0.5612403,
          "falsePositiveRate": 0.0002155637,
          "truePositiveCount": "1810",
          "falsePositiveCount": "1",
          "falseNegativeCount": "2829",
          "trueNegativeCount": "4638",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99106044,
          "recall": 0.38025436,
          "precision": 1.0,
          "f1Score": 0.5509917,
          "truePositiveCount": "1764",
          "falseNegativeCount": "2875",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99142385,
          "recall": 0.36990732,
          "precision": 1.0,
          "f1Score": 0.5400472,
          "truePositiveCount": "1716",
          "falseNegativeCount": "2923",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.991821,
          "recall": 0.35999137,
          "precision": 1.0,
          "f1Score": 0.52940243,
          "truePositiveCount": "1670",
          "falseNegativeCount": "2969",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99219143,
          "recall": 0.35007545,
          "precision": 1.0,
          "f1Score": 0.5186013,
          "truePositiveCount": "1624",
          "falseNegativeCount": "3015",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99265563,
          "recall": 0.3401595,
          "precision": 1.0,
          "f1Score": 0.50764036,
          "truePositiveCount": "1578",
          "falseNegativeCount": "3061",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9930375,
          "recall": 0.33045915,
          "precision": 1.0,
          "f1Score": 0.49675956,
          "truePositiveCount": "1533",
          "falseNegativeCount": "3106",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9932301,
          "recall": 0.32054323,
          "precision": 1.0,
          "f1Score": 0.48547176,
          "truePositiveCount": "1487",
          "falseNegativeCount": "3152",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99349546,
          "recall": 0.31041172,
          "precision": 1.0,
          "f1Score": 0.47376212,
          "truePositiveCount": "1440",
          "falseNegativeCount": "3199",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.993766,
          "recall": 0.30006468,
          "precision": 1.0,
          "f1Score": 0.461615,
          "truePositiveCount": "1392",
          "falseNegativeCount": "3247",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9940096,
          "recall": 0.29014874,
          "precision": 1.0,
          "f1Score": 0.44979113,
          "truePositiveCount": "1346",
          "falseNegativeCount": "3293",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9942971,
          "recall": 0.27980167,
          "precision": 1.0,
          "f1Score": 0.4372579,
          "truePositiveCount": "1298",
          "falseNegativeCount": "3341",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9945464,
          "recall": 0.26988575,
          "precision": 1.0,
          "f1Score": 0.42505518,
          "truePositiveCount": "1252",
          "falseNegativeCount": "3387",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.994734,
          "recall": 0.2601854,
          "precision": 1.0,
          "f1Score": 0.41293192,
          "truePositiveCount": "1207",
          "falseNegativeCount": "3432",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9949861,
          "recall": 0.25026944,
          "precision": 1.0,
          "f1Score": 0.40034482,
          "truePositiveCount": "1161",
          "falseNegativeCount": "3478",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99523896,
          "recall": 0.2399224,
          "precision": 1.0,
          "f1Score": 0.38699582,
          "truePositiveCount": "1113",
          "falseNegativeCount": "3526",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99545944,
          "recall": 0.23043759,
          "precision": 1.0,
          "f1Score": 0.37456203,
          "truePositiveCount": "1069",
          "falseNegativeCount": "3570",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9956529,
          "recall": 0.22009054,
          "precision": 1.0,
          "f1Score": 0.36077738,
          "truePositiveCount": "1021",
          "falseNegativeCount": "3618",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9958653,
          "recall": 0.21039017,
          "precision": 1.0,
          "f1Score": 0.34764025,
          "truePositiveCount": "976",
          "falseNegativeCount": "3663",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99604857,
          "recall": 0.20025867,
          "precision": 1.0,
          "f1Score": 0.33369252,
          "truePositiveCount": "929",
          "falseNegativeCount": "3710",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99617386,
          "recall": 0.19012718,
          "precision": 1.0,
          "f1Score": 0.31950733,
          "truePositiveCount": "882",
          "falseNegativeCount": "3757",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9963188,
          "recall": 0.18042682,
          "precision": 1.0,
          "f1Score": 0.3056976,
          "truePositiveCount": "837",
          "falseNegativeCount": "3802",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99644965,
          "recall": 0.17029533,
          "precision": 1.0,
          "f1Score": 0.29102966,
          "truePositiveCount": "790",
          "falseNegativeCount": "3849",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99657893,
          "recall": 0.16059496,
          "precision": 1.0,
          "f1Score": 0.27674592,
          "truePositiveCount": "745",
          "falseNegativeCount": "3894",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99672806,
          "recall": 0.1502479,
          "precision": 1.0,
          "f1Score": 0.2612444,
          "truePositiveCount": "697",
          "falseNegativeCount": "3942",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9968369,
          "recall": 0.14033197,
          "precision": 1.0,
          "f1Score": 0.24612476,
          "truePositiveCount": "651",
          "falseNegativeCount": "3988",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9969655,
          "recall": 0.13041604,
          "precision": 1.0,
          "f1Score": 0.23073989,
          "truePositiveCount": "605",
          "falseNegativeCount": "4034",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9970963,
          "recall": 0.12028454,
          "precision": 1.0,
          "f1Score": 0.21473928,
          "truePositiveCount": "558",
          "falseNegativeCount": "4081",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9972023,
          "recall": 0.110799745,
          "precision": 1.0,
          "f1Score": 0.19949543,
          "truePositiveCount": "514",
          "falseNegativeCount": "4125",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9973033,
          "recall": 0.100668244,
          "precision": 1.0,
          "f1Score": 0.18292205,
          "truePositiveCount": "467",
          "falseNegativeCount": "4172",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99741936,
          "recall": 0.09053675,
          "precision": 1.0,
          "f1Score": 0.16604072,
          "truePositiveCount": "420",
          "falseNegativeCount": "4219",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9974875,
          "recall": 0.080620825,
          "precision": 1.0,
          "f1Score": 0.14921205,
          "truePositiveCount": "374",
          "falseNegativeCount": "4265",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9976044,
          "recall": 0.070273764,
          "precision": 1.0,
          "f1Score": 0.13131924,
          "truePositiveCount": "326",
          "falseNegativeCount": "4313",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99769753,
          "recall": 0.06014227,
          "precision": 1.0,
          "f1Score": 0.11346076,
          "truePositiveCount": "279",
          "falseNegativeCount": "4360",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99778664,
          "recall": 0.049795214,
          "precision": 1.0,
          "f1Score": 0.09486653,
          "truePositiveCount": "231",
          "falseNegativeCount": "4408",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9978742,
          "recall": 0.040525977,
          "precision": 1.0,
          "f1Score": 0.07789517,
          "truePositiveCount": "188",
          "falseNegativeCount": "4451",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.997992,
          "recall": 0.030394481,
          "precision": 1.0,
          "f1Score": 0.058995817,
          "truePositiveCount": "141",
          "falseNegativeCount": "4498",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9980989,
          "recall": 0.020262988,
          "precision": 1.0,
          "f1Score": 0.03972111,
          "truePositiveCount": "94",
          "falseNegativeCount": "4545",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.99821925,
          "recall": 0.010347057,
          "precision": 1.0,
          "f1Score": 0.020482184,
          "truePositiveCount": "48",
          "falseNegativeCount": "4591",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 0.9987167,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4639",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        },
        {
          "confidenceThreshold": 1.0,
          "precision": "NaN",
          "f1Score": "NaN",
          "falseNegativeCount": "4639",
          "trueNegativeCount": "4639",
          "positionThreshold": 2147483647
        }
      ],
      "confusionMatrix": {
        "row": [
          {
            "exampleCount": [
              3941,
              147
            ]
          },
          {
            "exampleCount": [
              293,
              258
            ]
          }
        ],
        "displayName": [
          "1",
          "2"
        ]
      },
      "auRoc": 0.9743269,
      "logLoss": 0.20077285
    }
  }
]
```



### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

#### Call

In [ ]:
request = clients["tables"].get_model_evaluation(
    model_evaluation_name=evaluation_slice
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264/modelEvaluations/124023939996839080",
  "annotationSpecId": "not available",
  "createTime": "2021-02-28T17:24:31.800758Z",
  "evaluatedExampleCount": 4639,
  "classificationEvaluationMetrics": {
    "auPrc": 0.89604473,
    "confidenceMetricsEntry": [
      {
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.009267787,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.08771089,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.18867286,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.24711765,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.29667678,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.33910435,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.3722213,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.41141066,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.44597256,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.48088387,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.5,
        "recall": 0.90515196,
        "precision": 0.90515196,
        "f1Score": 0.90515196,
        "falsePositiveRate": 0.09484803,
        "truePositiveCount": "4199",
        "falsePositiveCount": "440",
        "falseNegativeCount": "440",
        "trueNegativeCount": "4199",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.51082355,
        "recall": 0.9006251,
        "precision": 0.90688086,
        "f1Score": 0.90374213,
        "falsePositiveRate": 0.09247683,
        "truePositiveCount": "4178",
        "falsePositiveCount": "429",
        "falseNegativeCount": "461",
        "trueNegativeCount": "4210",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.5395309,
        "recall": 0.8902781,
        "precision": 0.9145261,
        "f1Score": 0.9022392,
        "falsePositiveRate": 0.083207585,
        "truePositiveCount": "4130",
        "falsePositiveCount": "386",
        "falseNegativeCount": "509",
        "trueNegativeCount": "4253",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.569127,
        "recall": 0.88014656,
        "precision": 0.92083895,
        "f1Score": 0.90003306,
        "falsePositiveRate": 0.07566286,
        "truePositiveCount": "4083",
        "falsePositiveCount": "351",
        "falseNegativeCount": "556",
        "trueNegativeCount": "4288",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.59819055,
        "recall": 0.8700151,
        "precision": 0.9284564,
        "f1Score": 0.8982862,
        "falsePositiveRate": 0.06704031,
        "truePositiveCount": "4036",
        "falsePositiveCount": "311",
        "falseNegativeCount": "603",
        "trueNegativeCount": "4328",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.6267493,
        "recall": 0.86009914,
        "precision": 0.9350832,
        "f1Score": 0.8960252,
        "falsePositiveRate": 0.059711143,
        "truePositiveCount": "3990",
        "falsePositiveCount": "277",
        "falseNegativeCount": "649",
        "trueNegativeCount": "4362",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.6483357,
        "recall": 0.84996766,
        "precision": 0.9401526,
        "f1Score": 0.8927884,
        "falsePositiveRate": 0.05410649,
        "truePositiveCount": "3943",
        "falsePositiveCount": "251",
        "falseNegativeCount": "696",
        "trueNegativeCount": "4388",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.68742466,
        "recall": 0.8404829,
        "precision": 0.95051193,
        "f1Score": 0.8921176,
        "falsePositiveRate": 0.04375943,
        "truePositiveCount": "3899",
        "falsePositiveCount": "203",
        "falseNegativeCount": "740",
        "trueNegativeCount": "4436",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.7178758,
        "recall": 0.83035135,
        "precision": 0.95630586,
        "f1Score": 0.8888889,
        "falsePositiveRate": 0.03793921,
        "truePositiveCount": "3852",
        "falsePositiveCount": "176",
        "falseNegativeCount": "787",
        "trueNegativeCount": "4463",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.7454512,
        "recall": 0.8200043,
        "precision": 0.9620637,
        "f1Score": 0.8853718,
        "falsePositiveRate": 0.032334555,
        "truePositiveCount": "3804",
        "falsePositiveCount": "150",
        "falseNegativeCount": "835",
        "trueNegativeCount": "4489",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.7702336,
        "recall": 0.8105195,
        "precision": 0.9685729,
        "f1Score": 0.8825255,
        "falsePositiveRate": 0.02629877,
        "truePositiveCount": "3760",
        "falsePositiveCount": "122",
        "falseNegativeCount": "879",
        "trueNegativeCount": "4517",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.79181457,
        "recall": 0.80038804,
        "precision": 0.972244,
        "f1Score": 0.87798536,
        "falsePositiveRate": 0.022849752,
        "truePositiveCount": "3713",
        "falsePositiveCount": "106",
        "falseNegativeCount": "926",
        "trueNegativeCount": "4533",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.80905926,
        "recall": 0.7902565,
        "precision": 0.9755189,
        "f1Score": 0.873169,
        "falsePositiveRate": 0.01983186,
        "truePositiveCount": "3666",
        "falsePositiveCount": "92",
        "falseNegativeCount": "973",
        "trueNegativeCount": "4547",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.82475233,
        "recall": 0.7803406,
        "precision": 0.97574127,
        "f1Score": 0.86716974,
        "falsePositiveRate": 0.019400733,
        "truePositiveCount": "3620",
        "falsePositiveCount": "90",
        "falseNegativeCount": "1019",
        "trueNegativeCount": "4549",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.84347624,
        "recall": 0.7702091,
        "precision": 0.97863597,
        "f1Score": 0.86200243,
        "falsePositiveRate": 0.01681397,
        "truePositiveCount": "3573",
        "falsePositiveCount": "78",
        "falseNegativeCount": "1066",
        "trueNegativeCount": "4561",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.8612723,
        "recall": 0.7602932,
        "precision": 0.9799944,
        "f1Score": 0.8562758,
        "falsePositiveRate": 0.015520587,
        "truePositiveCount": "3527",
        "falsePositiveCount": "72",
        "falseNegativeCount": "1112",
        "trueNegativeCount": "4567",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.87450475,
        "recall": 0.75016165,
        "precision": 0.9811108,
        "f1Score": 0.8502321,
        "falsePositiveRate": 0.014442768,
        "truePositiveCount": "3480",
        "falsePositiveCount": "67",
        "falseNegativeCount": "1159",
        "trueNegativeCount": "4572",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.8876167,
        "recall": 0.74003017,
        "precision": 0.9825415,
        "f1Score": 0.8442149,
        "falsePositiveRate": 0.013149385,
        "truePositiveCount": "3433",
        "falsePositiveCount": "61",
        "falseNegativeCount": "1206",
        "trueNegativeCount": "4578",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.89729506,
        "recall": 0.7303298,
        "precision": 0.98402554,
        "f1Score": 0.8384063,
        "falsePositiveRate": 0.011856004,
        "truePositiveCount": "3388",
        "falsePositiveCount": "55",
        "falseNegativeCount": "1251",
        "trueNegativeCount": "4584",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9069939,
        "recall": 0.72041386,
        "precision": 0.9858407,
        "f1Score": 0.8324823,
        "falsePositiveRate": 0.010347057,
        "truePositiveCount": "3342",
        "falsePositiveCount": "48",
        "falseNegativeCount": "1297",
        "trueNegativeCount": "4591",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.91544837,
        "recall": 0.7102824,
        "precision": 0.987118,
        "f1Score": 0.8261251,
        "falsePositiveRate": 0.009269239,
        "truePositiveCount": "3295",
        "falsePositiveCount": "43",
        "falseNegativeCount": "1344",
        "trueNegativeCount": "4596",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9246296,
        "recall": 0.700582,
        "precision": 0.9896468,
        "f1Score": 0.8203963,
        "falsePositiveRate": 0.007329166,
        "truePositiveCount": "3250",
        "falsePositiveCount": "34",
        "falseNegativeCount": "1389",
        "trueNegativeCount": "4605",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.932719,
        "recall": 0.6900194,
        "precision": 0.99071497,
        "f1Score": 0.8134689,
        "falsePositiveRate": 0.006466911,
        "truePositiveCount": "3201",
        "falsePositiveCount": "30",
        "falseNegativeCount": "1438",
        "trueNegativeCount": "4609",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.93940556,
        "recall": 0.6807502,
        "precision": 0.9921458,
        "f1Score": 0.80746615,
        "falsePositiveRate": 0.0053890925,
        "truePositiveCount": "3158",
        "falsePositiveCount": "25",
        "falseNegativeCount": "1481",
        "trueNegativeCount": "4614",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9436984,
        "recall": 0.6704031,
        "precision": 0.99234205,
        "f1Score": 0.8002058,
        "falsePositiveRate": 0.0051735286,
        "truePositiveCount": "3110",
        "falsePositiveCount": "24",
        "falseNegativeCount": "1529",
        "trueNegativeCount": "4615",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9486017,
        "recall": 0.6602716,
        "precision": 0.992547,
        "f1Score": 0.7930097,
        "falsePositiveRate": 0.004957965,
        "truePositiveCount": "3063",
        "falsePositiveCount": "23",
        "falseNegativeCount": "1576",
        "trueNegativeCount": "4616",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.95302504,
        "recall": 0.6501401,
        "precision": 0.9937397,
        "f1Score": 0.78603077,
        "falsePositiveRate": 0.0040957103,
        "truePositiveCount": "3016",
        "falsePositiveCount": "19",
        "falseNegativeCount": "1623",
        "trueNegativeCount": "4620",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.95584136,
        "recall": 0.6400086,
        "precision": 0.99530673,
        "f1Score": 0.7790606,
        "falsePositiveRate": 0.003017892,
        "truePositiveCount": "2969",
        "falsePositiveCount": "14",
        "falseNegativeCount": "1670",
        "trueNegativeCount": "4625",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.95881057,
        "recall": 0.6300927,
        "precision": 0.9955722,
        "f1Score": 0.7717492,
        "falsePositiveRate": 0.002802328,
        "truePositiveCount": "2923",
        "falsePositiveCount": "13",
        "falseNegativeCount": "1716",
        "trueNegativeCount": "4626",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.96166563,
        "recall": 0.62082344,
        "precision": 0.9961951,
        "f1Score": 0.76494026,
        "falsePositiveRate": 0.0023712006,
        "truePositiveCount": "2880",
        "falsePositiveCount": "11",
        "falseNegativeCount": "1759",
        "trueNegativeCount": "4628",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.96420497,
        "recall": 0.61026084,
        "precision": 0.9961295,
        "f1Score": 0.75685066,
        "falsePositiveRate": 0.0023712006,
        "truePositiveCount": "2831",
        "falsePositiveCount": "11",
        "falseNegativeCount": "1808",
        "trueNegativeCount": "4628",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.96650803,
        "recall": 0.5999138,
        "precision": 0.996063,
        "f1Score": 0.7488228,
        "falsePositiveRate": 0.0023712006,
        "truePositiveCount": "2783",
        "falsePositiveCount": "11",
        "falseNegativeCount": "1856",
        "trueNegativeCount": "4628",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.96897614,
        "recall": 0.59042895,
        "precision": 0.9963623,
        "f1Score": 0.74147266,
        "falsePositiveRate": 0.002155637,
        "truePositiveCount": "2739",
        "falsePositiveCount": "10",
        "falseNegativeCount": "1900",
        "trueNegativeCount": "4629",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.97092766,
        "recall": 0.5807286,
        "precision": 0.99667037,
        "f1Score": 0.73385996,
        "falsePositiveRate": 0.0019400733,
        "truePositiveCount": "2694",
        "falsePositiveCount": "9",
        "falseNegativeCount": "1945",
        "trueNegativeCount": "4630",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9727514,
        "recall": 0.5699504,
        "precision": 0.9969834,
        "f1Score": 0.7252777,
        "falsePositiveRate": 0.0017245096,
        "truePositiveCount": "2644",
        "falsePositiveCount": "8",
        "falseNegativeCount": "1995",
        "trueNegativeCount": "4631",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9747563,
        "recall": 0.56046563,
        "precision": 0.99731493,
        "f1Score": 0.7176373,
        "falsePositiveRate": 0.001508946,
        "truePositiveCount": "2600",
        "falsePositiveCount": "7",
        "falseNegativeCount": "2039",
        "trueNegativeCount": "4632",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9766838,
        "recall": 0.549903,
        "precision": 0.9976535,
        "f1Score": 0.709005,
        "falsePositiveRate": 0.0012933821,
        "truePositiveCount": "2551",
        "falsePositiveCount": "6",
        "falseNegativeCount": "2088",
        "trueNegativeCount": "4633",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.97776866,
        "recall": 0.54063374,
        "precision": 0.99801034,
        "f1Score": 0.7013423,
        "falsePositiveRate": 0.0010778185,
        "truePositiveCount": "2508",
        "falsePositiveCount": "5",
        "falseNegativeCount": "2131",
        "trueNegativeCount": "4634",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9792647,
        "recall": 0.53050226,
        "precision": 0.9979724,
        "f1Score": 0.6927516,
        "falsePositiveRate": 0.0010778185,
        "truePositiveCount": "2461",
        "falsePositiveCount": "5",
        "falseNegativeCount": "2178",
        "trueNegativeCount": "4634",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9804138,
        "recall": 0.5203708,
        "precision": 0.99834573,
        "f1Score": 0.6841434,
        "falsePositiveRate": 0.0008622548,
        "truePositiveCount": "2414",
        "falsePositiveCount": "4",
        "falseNegativeCount": "2225",
        "trueNegativeCount": "4635",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9814273,
        "recall": 0.51045483,
        "precision": 0.9991561,
        "f1Score": 0.6757027,
        "falsePositiveRate": 0.0004311274,
        "truePositiveCount": "2368",
        "falsePositiveCount": "2",
        "falseNegativeCount": "2271",
        "trueNegativeCount": "4637",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.98243207,
        "recall": 0.5005389,
        "precision": 0.9991394,
        "f1Score": 0.6669539,
        "falsePositiveRate": 0.0004311274,
        "truePositiveCount": "2322",
        "falsePositiveCount": "2",
        "falseNegativeCount": "2317",
        "trueNegativeCount": "4637",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9835001,
        "recall": 0.49019185,
        "precision": 0.9995604,
        "f1Score": 0.6577958,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "2274",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2365",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9844133,
        "recall": 0.4804915,
        "precision": 0.9995516,
        "f1Score": 0.6490028,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "2229",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2410",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.98528725,
        "recall": 0.47079113,
        "precision": 0.99954236,
        "f1Score": 0.6400938,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "2184",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2455",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.98630506,
        "recall": 0.46001294,
        "precision": 0.9995316,
        "f1Score": 0.6300561,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "2134",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2505",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9871925,
        "recall": 0.44988143,
        "precision": 0.9995211,
        "f1Score": 0.6204846,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "2087",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2552",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9877509,
        "recall": 0.44082776,
        "precision": 0.99951124,
        "f1Score": 0.6118175,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "2045",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2594",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9885219,
        "recall": 0.43048072,
        "precision": 0.9994995,
        "f1Score": 0.6017779,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "1997",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2642",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9892192,
        "recall": 0.4199181,
        "precision": 0.9994869,
        "f1Score": 0.5913783,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "1948",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2691",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9897657,
        "recall": 0.41000214,
        "precision": 0.9994745,
        "f1Score": 0.5814735,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "1902",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2737",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9902156,
        "recall": 0.40051734,
        "precision": 0.99946207,
        "f1Score": 0.57186824,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "1858",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2781",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.990671,
        "recall": 0.3901703,
        "precision": 0.9994478,
        "f1Score": 0.5612403,
        "falsePositiveRate": 0.0002155637,
        "truePositiveCount": "1810",
        "falsePositiveCount": "1",
        "falseNegativeCount": "2829",
        "trueNegativeCount": "4638",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99106044,
        "recall": 0.38025436,
        "precision": 1.0,
        "f1Score": 0.5509917,
        "truePositiveCount": "1764",
        "falseNegativeCount": "2875",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99142385,
        "recall": 0.36990732,
        "precision": 1.0,
        "f1Score": 0.5400472,
        "truePositiveCount": "1716",
        "falseNegativeCount": "2923",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.991821,
        "recall": 0.35999137,
        "precision": 1.0,
        "f1Score": 0.52940243,
        "truePositiveCount": "1670",
        "falseNegativeCount": "2969",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99219143,
        "recall": 0.35007545,
        "precision": 1.0,
        "f1Score": 0.5186013,
        "truePositiveCount": "1624",
        "falseNegativeCount": "3015",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99265563,
        "recall": 0.3401595,
        "precision": 1.0,
        "f1Score": 0.50764036,
        "truePositiveCount": "1578",
        "falseNegativeCount": "3061",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9930375,
        "recall": 0.33045915,
        "precision": 1.0,
        "f1Score": 0.49675956,
        "truePositiveCount": "1533",
        "falseNegativeCount": "3106",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9932301,
        "recall": 0.32054323,
        "precision": 1.0,
        "f1Score": 0.48547176,
        "truePositiveCount": "1487",
        "falseNegativeCount": "3152",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99349546,
        "recall": 0.31041172,
        "precision": 1.0,
        "f1Score": 0.47376212,
        "truePositiveCount": "1440",
        "falseNegativeCount": "3199",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.993766,
        "recall": 0.30006468,
        "precision": 1.0,
        "f1Score": 0.461615,
        "truePositiveCount": "1392",
        "falseNegativeCount": "3247",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9940096,
        "recall": 0.29014874,
        "precision": 1.0,
        "f1Score": 0.44979113,
        "truePositiveCount": "1346",
        "falseNegativeCount": "3293",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9942971,
        "recall": 0.27980167,
        "precision": 1.0,
        "f1Score": 0.4372579,
        "truePositiveCount": "1298",
        "falseNegativeCount": "3341",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9945464,
        "recall": 0.26988575,
        "precision": 1.0,
        "f1Score": 0.42505518,
        "truePositiveCount": "1252",
        "falseNegativeCount": "3387",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.994734,
        "recall": 0.2601854,
        "precision": 1.0,
        "f1Score": 0.41293192,
        "truePositiveCount": "1207",
        "falseNegativeCount": "3432",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9949861,
        "recall": 0.25026944,
        "precision": 1.0,
        "f1Score": 0.40034482,
        "truePositiveCount": "1161",
        "falseNegativeCount": "3478",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99523896,
        "recall": 0.2399224,
        "precision": 1.0,
        "f1Score": 0.38699582,
        "truePositiveCount": "1113",
        "falseNegativeCount": "3526",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99545944,
        "recall": 0.23043759,
        "precision": 1.0,
        "f1Score": 0.37456203,
        "truePositiveCount": "1069",
        "falseNegativeCount": "3570",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9956529,
        "recall": 0.22009054,
        "precision": 1.0,
        "f1Score": 0.36077738,
        "truePositiveCount": "1021",
        "falseNegativeCount": "3618",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9958653,
        "recall": 0.21039017,
        "precision": 1.0,
        "f1Score": 0.34764025,
        "truePositiveCount": "976",
        "falseNegativeCount": "3663",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99604857,
        "recall": 0.20025867,
        "precision": 1.0,
        "f1Score": 0.33369252,
        "truePositiveCount": "929",
        "falseNegativeCount": "3710",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99617386,
        "recall": 0.19012718,
        "precision": 1.0,
        "f1Score": 0.31950733,
        "truePositiveCount": "882",
        "falseNegativeCount": "3757",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9963188,
        "recall": 0.18042682,
        "precision": 1.0,
        "f1Score": 0.3056976,
        "truePositiveCount": "837",
        "falseNegativeCount": "3802",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99644965,
        "recall": 0.17029533,
        "precision": 1.0,
        "f1Score": 0.29102966,
        "truePositiveCount": "790",
        "falseNegativeCount": "3849",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99657893,
        "recall": 0.16059496,
        "precision": 1.0,
        "f1Score": 0.27674592,
        "truePositiveCount": "745",
        "falseNegativeCount": "3894",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99672806,
        "recall": 0.1502479,
        "precision": 1.0,
        "f1Score": 0.2612444,
        "truePositiveCount": "697",
        "falseNegativeCount": "3942",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9968369,
        "recall": 0.14033197,
        "precision": 1.0,
        "f1Score": 0.24612476,
        "truePositiveCount": "651",
        "falseNegativeCount": "3988",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9969655,
        "recall": 0.13041604,
        "precision": 1.0,
        "f1Score": 0.23073989,
        "truePositiveCount": "605",
        "falseNegativeCount": "4034",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9970963,
        "recall": 0.12028454,
        "precision": 1.0,
        "f1Score": 0.21473928,
        "truePositiveCount": "558",
        "falseNegativeCount": "4081",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9972023,
        "recall": 0.110799745,
        "precision": 1.0,
        "f1Score": 0.19949543,
        "truePositiveCount": "514",
        "falseNegativeCount": "4125",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9973033,
        "recall": 0.100668244,
        "precision": 1.0,
        "f1Score": 0.18292205,
        "truePositiveCount": "467",
        "falseNegativeCount": "4172",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99741936,
        "recall": 0.09053675,
        "precision": 1.0,
        "f1Score": 0.16604072,
        "truePositiveCount": "420",
        "falseNegativeCount": "4219",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9974875,
        "recall": 0.080620825,
        "precision": 1.0,
        "f1Score": 0.14921205,
        "truePositiveCount": "374",
        "falseNegativeCount": "4265",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9976044,
        "recall": 0.070273764,
        "precision": 1.0,
        "f1Score": 0.13131924,
        "truePositiveCount": "326",
        "falseNegativeCount": "4313",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99769753,
        "recall": 0.06014227,
        "precision": 1.0,
        "f1Score": 0.11346076,
        "truePositiveCount": "279",
        "falseNegativeCount": "4360",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99778664,
        "recall": 0.049795214,
        "precision": 1.0,
        "f1Score": 0.09486653,
        "truePositiveCount": "231",
        "falseNegativeCount": "4408",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9978742,
        "recall": 0.040525977,
        "precision": 1.0,
        "f1Score": 0.07789517,
        "truePositiveCount": "188",
        "falseNegativeCount": "4451",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.997992,
        "recall": 0.030394481,
        "precision": 1.0,
        "f1Score": 0.058995817,
        "truePositiveCount": "141",
        "falseNegativeCount": "4498",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9980989,
        "recall": 0.020262988,
        "precision": 1.0,
        "f1Score": 0.03972111,
        "truePositiveCount": "94",
        "falseNegativeCount": "4545",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.99821925,
        "recall": 0.010347057,
        "precision": 1.0,
        "f1Score": 0.020482184,
        "truePositiveCount": "48",
        "falseNegativeCount": "4591",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 0.9987167,
        "precision": "NaN",
        "f1Score": "NaN",
        "falseNegativeCount": "4639",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      },
      {
        "confidenceThreshold": 1.0,
        "precision": "NaN",
        "f1Score": "NaN",
        "falseNegativeCount": "4639",
        "trueNegativeCount": "4639",
        "positionThreshold": 1
      }
    ],
    "confusionMatrix": {
      "row": [
        {
          "exampleCount": [
            3941,
            147
          ]
        },
        {
          "exampleCount": [
            293,
            258
          ]
        }
      ],
      "displayName": [
        "1",
        "2"
      ]
    },
    "auRoc": 0.94022393,
    "logLoss": 0.20077285
  }
}
```



## Make batch predictions

### Make a batch prediction file



In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = "gs://" + BUCKET_NAME + "/batch.csv"

! gsutil cp batch.csv $gcs_input_uri
! gsutil cat $gcs_input_uri


*Example output*:
```
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome
53,management,married,tertiary,no,583,no,no,cellular,17,nov,226,1,184,4,success
34,admin.,single,secondary,no,557,no,no,cellular,17,nov,224,1,-1,0,unknown
23,student,single,tertiary,no,113,no,no,cellular,17,nov,266,1,-1,0,unknown
73,retired,married,secondary,no,2850,no,no,cellular,17,nov,300,1,40,8,failure
25,technician,single,secondary,no,505,no,yes,cellular,17,nov,386,2,-1,0,unknown
51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown
71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown
72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success
57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown
37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other
```



### [projects.locations.models.batchPredict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

#### Request

In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [gcs_input_uri]
    }
}

output_config = {
    "gcs_destination": {
        "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
    }
}

batch_prediction = automl.BatchPredictRequest(
    name=model_id,
    input_config=input_config,
    output_config=output_config
)

print(MessageToJson(
    batch_prediction.__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210228161105/batch.csv"
      ]
    }
  },
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210228161105/batch_output/"
    }
  }
}
```



#### Call

In [ ]:
request = clients["tables"].batch_predict(
    model_name=model_id, 
    gcs_input_uris=[gcs_input_uri],
    gcs_output_uri_prefix="gs://" + f"{BUCKET_NAME}/batch_output/"
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{}
```



In [ ]:
destination_uri = batch_prediction.output_config.gcs_destination.output_uri_prefix[:-1]

! gsutil ls $destination_uri/*
! gsutil cat $destination_uri/prediction*/*.csv


*Example output*:
```
gs://migration-ucaip-trainingaip-20210228161105/batch_output/prediction-bank_20210228161105-2021-02-28T17:28:50.618755Z/errors_1.csv
gs://migration-ucaip-trainingaip-20210228161105/batch_output/prediction-bank_20210228161105-2021-02-28T17:28:50.618755Z/tables_1.csv
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome,errors_Deposit
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome,Deposit_1_score,Deposit_2_score
73,retired,married,secondary,no,2850,no,no,cellular,17,nov,300,1,40,8,failure,0.55540502071380615,0.44459491968154907
72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,0.29455980658531189,0.7054402232170105
51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,0.478891521692276,0.52110844850540161
53,management,married,tertiary,no,583,no,no,cellular,17,nov,226,1,184,4,success,0.36687871813774109,0.63312125205993652
23,student,single,tertiary,no,113,no,no,cellular,17,nov,266,1,-1,0,unknown,0.62017822265625,0.37982171773910522
37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,0.78739255666732788,0.21260741353034973
25,technician,single,secondary,no,505,no,yes,cellular,17,nov,386,2,-1,0,unknown,0.7512086033821106,0.24879136681556702
57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,0.86376494169235229,0.13623508810997009
71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,0.47541293501853943,0.52458703517913818
34,admin.,single,secondary,no,557,no,no,cellular,17,nov,224,1,-1,0,unknown,0.90285629034042358,0.097143754363059998
```



## Make online predictions

### [projects.locations.models.deploy](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/deploy)

#### Call

In [ ]:
request = clients["tables"].deploy_model(
    model_name=model_id
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```



### [projects.locations.models.predict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/predict)

#### Prepare data item for online prediction

In [ ]:
INSTANCE = {
    "Age": '58', 
    "Job": "managment", 
    "MaritalStatus": "married", 
    "Education": "teritary", 
    "Default": "no",
    "Balance": '2143', 
    "Housing": "yes", 
    "Loan": "no", 
    "Contact": "unknown", 
    "Day": '5', 
    "Month": "may",
    "Duration": '261', 
    "Campaign": '1', 
    "PDays": '-1', 
    "Previous": 0, 
    "POutcome": "unknown"
}

#### Request

In [ ]:
instances_list = [INSTANCE]
instances = [ParseDict(s, Value()) for s in instances_list]

print(MessageToJson(
    automl.PredictRequest(
        name=model_id,
        payload={"row": {"values": instances}}            
    ).__dict__["_pb"])
)

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TBL8310014324731019264",
  "payload": {
    "row": {
      "values": [
        {
          "Duration": "261",
          "MaritalStatus": "married",
          "Loan": "no",
          "Education": "teritary",
          "Default": "no",
          "Campaign": "1",
          "Balance": "2143",
          "Contact": "unknown",
          "Previous": 0.0,
          "Day": "5",
          "Housing": "yes",
          "PDays": "-1",
          "Job": "managment",
          "Month": "may",
          "Age": "58",
          "POutcome": "unknown"
        }
      ]
    }
  }
}
```



#### Call

In [ ]:
request = clients["tables"].predict(
    model_name=model_id,
    inputs=INSTANCE
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "payload": [
    {
      "tables": {
        "score": 0.9947612,
        "value": "1"
      }
    },
    {
      "tables": {
        "score": 0.005238751,
        "value": "2"
      }
    }
  ]
}
```



### [projects.locations.models.undeploy](https://cloud.google.com/automl/docs/reference/rest/v1/projects.locations.models/undeploy)

#### Request

In [ ]:
request = clients["automl"].undeploy_model(
    name=model_id
)


#### Call

In [ ]:
result = request.result()


#### Response



In [ ]:
print(MessageToJson(result))


*Example output*:

```
{}
```



# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_model = True
delete_bucket = True

# Delete the dataset using the AutoML fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['automl'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients['automl'].delete_model(name=model_id)
except Exception as e:
    print(e)


if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
